This analysis was performed succesfully in "CentOS Linux release 7.5.1804 (Core)". If you use different version of Linux or OS, you may need to adjust it accordingly and we are not responsible for any liability or error.

Most of the figures were post-processed using Adobe Illustrator to allow for easier adjustment of font-size, color, lines, etc.

Other information and issue reporting, please refer to https://github.com/muharif/PFProgression_Analysis

## Init

In [ ]:
%matplotlib inline
import pandas as pd
import rpy2,os,re, shutil
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import seaborn as sns
import copy
sns.set(font="Arial")
sns.set_style("white")
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
from sklearn.decomposition import PCA as sklearnPCA
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import pickle
from statannot import add_stat_annotation
from matplotlib.gridspec import GridSpec


#rpy2
import rpy2.robjects as robjects
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
from rpy2.robjects import pandas2ri
pandas2ri.activate()
import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore", category=RRuntimeWarning)

from scipy.stats import zscore, spearmanr, ttest_ind, pearsonr,hypergeom, f_oneway
from scipy.interpolate import interp1d
from statsmodels.sandbox.stats.multicomp import multipletests
import leidenalg
import igraph as ig
import gseapy as gp
from scipy.spatial import distance
from scipy.cluster.hierarchy import dendrogram, linkage, cut_tree
from sklearn.impute import KNNImputer
from scipy.io import loadmat
import itertools

## Functions

In [ ]:
class DESeq_Piano:
    def __init__(self,respath='.'):
        if respath=='.':
            print('Result Path is Not Defined, any result will be written in this directory')
            self.respath=respath
        elif os.path.isdir(respath):
            print('Result Path Exists, please be careful of overwriting')
            self.respath=respath
        else:
            print('Result Path Does Not Exists, creating new directory')
            os.mkdir(respath)
            self.respath=respath
        self.__Init()
        
    def __Init(self):
        robjects.r('''
            library('DESeq2')
            library('piano')
            library('Biobase')
            library('snow')
            library('RColorBrewer')
            library('gplots')
            library('snowfall')
        ''')
        self.GSC_KEGG=None
        self.GSC_TF=None
        self.GSC_RM=None
        self.GSC_GO=None
        self.GSC_extraGSC=None
        self.KEGG=None
        self.TF=None
        self.RM=None
        self.GO=None
        self.extraGSC=None
        self.dds=None
        self.deseq_res=None
        self.conds=None
        self.count=None
        self.tpm=None

    def __check_dir(self,extrapath):
        if os.path.isdir('%s/%s/' % (self.respath,extrapath)):
            print('Path %s/%s/ exist, writing in the folder' % (self.respath,extrapath))
        else:
            print('Path %s/%s/ does not exist, making the folder to write the results' % (self.respath,extrapath))
            os.mkdir('%s/%s/' % (self.respath,extrapath))
            
    def DEseq(self,count_df=None,conds_series=None):
        if count_df is not None:
            self.count=count_df
        if conds_series is not None:
            self.conds=conds_series
        self.gene_names=count_df.index
        robjects.globalenv['conds']=np.array(self.conds)
        robjects.globalenv['subject']=np.array(self.conds.index)
        robjects.globalenv['count']=self.count.astype(int).values
        robjects.globalenv['gene_names']=np.array(self.gene_names)
        robjects.r('''
            conds=as.factor(conds)
            coldata <- data.frame(row.names=subject,conds)
            dds <- DESeqDataSetFromMatrix(countData=as.matrix(count),colData=coldata,design=~conds)
            dds <- DESeq(dds)
            print(resultsNames(dds))
            normalized_counts = counts(dds,normalized=TRUE)
            rownames(normalized_counts) = gene_names
            write.table(normalized_counts, file="../Data/normalized_counts.txt", sep="\t", quote=F, col.names=NA)
        ''')
        self.dds=robjects.globalenv['dds']
    
    def DEseq_Compare(self, cond1, cond2,save=True):
        if self.dds is None:
            raise ValueError('No DESeq data found, run the DEseq function')
        else:
            print('Comparing %s and %s' % (cond1,cond2))
            robjects.globalenv['dds']=self.dds
            robjects.globalenv['cond1']=cond1
            robjects.globalenv['cond2']=cond2
            robjects.globalenv['gene_names']=np.array(self.gene_names)
            robjects.r('''
                res=results(dds,contrast=c('conds',cond1,cond2))
                res=data.frame(res)
                res$rownames=gene_names
            ''')
            res = pd.DataFrame(robjects.globalenv['res']).set_index('rownames')
            self.deseq_df=res
            if save:
                self.__check_dir('deseq')
                res.to_csv('%s/deseq/deseq_%s_%s.txt' % (self.respath,cond1,cond2),sep='\t')
        
    def loadGSC(self,KEGG='',TF='',RM='',GO='',extraGSC=''):
        if KEGG != '':
            name='KEGG'
            self.__check_dir(name)
            self.GSC_KEGG=self.__loadGSC_execute(KEGG)
        if GO != '':
            name='GO'
            self.__check_dir(name)
            self.GSC_GO=self.__loadGSC_execute(GO)
    
    def __loadGSC_execute(self,GSC):
        robjects.globalenv['GSC']=GSC 
        robjects.r('''
            y=loadGSC(GSC)
        ''')
        return robjects.globalenv['y']
    
    def __PIANO_execute(self,cond1,cond2,GSCtype='',save=True):
        robjects.globalenv['deseq_file']=pandas2ri.py2rpy_pandasdataframe(self.deseq_df)
        if GSCtype == 'KEGG':
            robjects.globalenv['y']=self.GSC_KEGG
            deseq_df=self.deseq_df
            deseq_df.index=[i.upper() for i in deseq_df.index]
            robjects.globalenv['deseq_file']=pandas2ri.py2rpy_pandasdataframe(deseq_df)
        elif GSCtype == 'GO':
            robjects.globalenv['y']=self.GSC_GO
            deseq_df=self.deseq_df
            deseq_df.index=[i.upper() for i in deseq_df.index]
            robjects.globalenv['deseq_file']=pandas2ri.py2rpy_pandasdataframe(deseq_df)
        robjects.r('''
            DESeq_file=deseq_file
            DESeq_file=DESeq_file[ ,c('log2FoldChange','pvalue')]
            logFC=as.matrix(DESeq_file[,1])
            pval=as.matrix(DESeq_file[,2])
            rownames(logFC)=(rownames(DESeq_file))
            rownames(pval)=(rownames(DESeq_file))
            logFC[is.na(logFC)] <- 0
            pval[is.na(pval)] <- 1
            gsaRes <- runGSA(pval,logFC,gsc=y, geneSetStat="reporter", signifMethod="nullDist", nPerm=1000,ncpus=8,gsSizeLim = c(5, Inf))
            res_piano=GSAsummaryTable(gsaRes)
            res_piano$rownames=rownames(res_piano)
        ''')
        res=pd.DataFrame(robjects.globalenv['res_piano']).set_index('rownames').set_index('Name').iloc[0:,0:]#.groupby('Name').sum()
        if save:
            res.to_csv('%s/%s/piano_%s_%s.txt' % (self.respath,GSCtype,cond1,cond2),sep='\t')
        return res

    def PIANO(self,cond1,cond2,save=True):
        if self.deseq_df is None:
            raise ValueError('No DEseq comparison found, run DEseq_Compare first or assign deseq result dataframe to deseq_res')
        else:
            if self.GSC_KEGG is not None:
                name='KEGG'
                self.KEGG = self.__PIANO_execute(cond1,cond2,GSCtype=name,save=save)
            if self.GSC_GO is not None:
                name='GO'
                self.GO = self.__PIANO_execute(cond1,cond2,GSCtype=name,save=save)         
                    
    def __piano2xlsx(self,name,part='ALL'):
        if part != 'ALL':
            files=[i for i in sorted(os.listdir('%s/%s' % (self.respath,name))) if part in i]
        else:
            files=sorted(os.listdir('%s/%s' % (self.respath,name)))
        writer = pd.ExcelWriter('%s/%s_%s.xlsx' % (self.respath,name,part), engine='xlsxwriter')
        for i in files:
            if i.startswith('.'):
                continue
            deseq1=pd.read_csv('%s/%s/%s' % (self.respath,name,i),index_col='Name',sep='\t')[['Genes (tot)','Stat (dist.dir.up)','p adj (dist.dir.up)']]
            deseq1.columns=['# of Genes', 'Stats','P-Adj']
            deseq1=deseq1[deseq1['P-Adj']<0.05]
            deseq1['Direction']='UP'
            deseq2=pd.read_csv('%s/%s/%s' % (self.respath,name,i),index_col='Name',sep='\t')[['Genes (tot)','Stat (dist.dir.up)','p adj (dist.dir.dn)']]
            deseq2.columns=['# of Genes', 'Stats','P-Adj']
            deseq2=deseq2[deseq2['P-Adj']<0.05]
            deseq2['Direction']='DOWN'
            deseq1=pd.concat([deseq1,deseq2])
            if len(i.replace('piano_','').replace('.txt',''))>31:
                print('sheet name too big, trimming to 30 character')
                n_temp=i.replace('piano_','').replace('.txt','')[0:31]
            else:
                n_temp=i.replace('piano_','').replace('.txt','')
            deseq1.sort_values('P-Adj').to_excel(writer, sheet_name=n_temp)
        writer.save()
        
    def summarizePiano(self,names=['KEGG','GO','TF','RM','extraGSC'],part='ALL'):
        for name in names:
            if os.path.isdir(self.respath+name):
                self.__piano2xlsx(name,part=part)
            else:
                continue
    
    def summarizeDEseq(self,deseq_result='deseq',part='ALL'):
        if os.path.isdir(self.respath+deseq_result):
            if part != 'ALL':
                files=[i for i in sorted(os.listdir(self.respath+'/deseq/')) if part in i]
            else:
                files=sorted(os.listdir(self.respath+'/deseq/'))
            writer = pd.ExcelWriter(self.respath+'/DifferentialExpression_%s.xlsx' % part, engine='xlsxwriter')
            fin=0
            for i in files:
                if i.startswith('.'):
                    continue
                deseq=pd.read_csv(self.respath+'/deseq/%s' % i,index_col=0,sep='\t')
                deseq['abs']=deseq['log2FoldChange'].abs()
                deseq=deseq.sort_values('abs',ascending=False)[['log2FoldChange','pvalue','padj']]

                deseq['Direction']=['UP' if i > 0 else 'DOWN' for i in deseq['log2FoldChange']]
                deseq.columns=['L2FC','P-VALUE','P-ADJ','DIRECTION']
                deseq=deseq.sort_values('P-VALUE')
                if len(i.replace('deseq_','').replace('.txt',''))>31:
                    print('sheet name too big, trimming to 30 character')
                    n_temp=i.replace('deseq_','').replace('.txt','')[0:31]
                else:
                    n_temp=i.replace('deseq_','').replace('.txt','')
                deseq.to_excel(writer, sheet_name=n_temp)
            writer.save()
        else:
            raise ValueError('No DESEq result found')

    def save_object(self,filename='DEseq_PIANO.pkl'):
        with open('%s/%s' % (self.respath,filename), 'wb') as file:
            pickle.dump(self, file)
    
    def load_object(self,filename='DEseq_PIANO.pkl'):
        with open(filename, 'rb') as file:
            res=pickle.load(file)
        return res

def filter_tpm(tpm,conds,thr=1):
    #making sure that in at least 1 condition we have TPM > threshold (default 1)
    return tpm[(pd.concat([tpm.T,conds],1).groupby(conds.name).mean().T>thr).sum(1)>0]

def calc(tpm, clin = None,pval_thr=0.05,pos_only=False):
    print('Calculating Correlation..')
    if (clin) == None:
        tpm = tpm
    else:   
        tpm = pd.concat([tpm.T,clin], 1).T
        temp=spearmanr(tpm.T)
        corr=pd.DataFrame(temp[0],columns=list(tpm.index),index=list(tpm.index))
        pval=pd.DataFrame(temp[1],columns=list(tpm.index),index=list(tpm.index))
        print('Filtering the matrix Correlation..')
        corr=corr.where(np.triu(np.ones(corr.shape)).astype(np.bool))
        pval=pval.where(np.triu(np.ones(pval.shape)).astype(np.bool))
        print('Making long table of Correlation..')
        corr2=corr.unstack().reset_index(name='weight')
        pval2=pval.unstack().reset_index(name='pval')
        res=corr2.merge(pval2,on=['level_0','level_1'])
        res=res[res['level_0'] != res['level_1']]
        res=res.dropna()
        print('Adjusting P-val')
        res['padj']=multipletests(res['pval'],method='fdr_bh')[1]
        res=res[res.padj < pval_thr].reset_index(drop=True)
        if pos_only:
            res=res[res['weight']>0]
        res=res[['level_0','level_1','weight']]
        print('Done!!')
    return res


class Network_Analysis:
    def __init__(self,filename, network_type,respath,KEGG='',GO='',topx=0.05,cluster_size=30):
        self.network_ori=pd.read_csv(filename,sep='\t')
        self.res_path=respath
        if os.path.isdir(self.res_path):
            pass
        else:
            os.mkdir(self.res_path)
        self.cluster_size=cluster_size
        self.topx=topx
        if network_type == 'transcriptomics':
            self.__net_analysis_transcriptomics()
        else:
            self.__net_analysis_otheromics()
        
    def __net_analysis_transcriptomics(self):
        print('Loading The Network...')
        temp=self.network_ori[self.network_ori['weight']>0]
        temp=temp[temp['weight']>temp['weight'].quantile(1-self.topx)]
        g= ig.Graph.TupleList(zip(temp['level_0'],temp['level_1']))
        print('Cluster Analysis...')
        optimiser = leidenalg.Optimiser()
        clust_calc = leidenalg.ModularityVertexPartition(g)
        diff =  optimiser.optimise_partition(clust_calc, n_iterations=-1)
        clust=pd.Series(clust_calc.membership,index=g.vs['name'])
        thr=self.cluster_size
        temp_c=clust.value_counts()[clust.value_counts()>thr].index.tolist()
        self.degree = pd.Series(g.degree(), index = g.vs['name'], name = 'degree')
        self.clustering=clust[clust.isin(temp_c)]
        self.modularity=clust_calc.modularity
        self.network=g
        temp=self.network_ori.merge(pd.DataFrame(self.clustering).reset_index(),how='left',left_on='level_0',right_on='index').merge(pd.DataFrame(self.clustering).reset_index(),how='left',left_on='level_1',right_on='index').dropna()
        temp2=temp[['0_x','0_y']]
        temp2['count']=1
        temp2=temp2.groupby(['0_x','0_y']).count().reset_index().pivot_table(index='0_x',columns='0_y',values='count').fillna(0)
        temp2=pd.DataFrame([(i,j,(temp2.loc[i,j]+temp2.loc[j,i]),(temp2.loc[i,j]+temp2.loc[j,i])/(self.clustering.value_counts()[i]+self.clustering.value_counts()[j])) for i in temp2.index for j in temp2.columns if (i < j)],columns=['Source','Target','Weight1','Weight2'])
        temp2.astype(int).to_csv('%s/edge_pos_cluster.txt' % self.res_path,sep='\t')
        pd.DataFrame(self.clustering.value_counts(),columns=['Size']).to_csv('%s/node_pos_cluster.txt' % self.res_path,sep='\t')
        pd.concat([self.clustering.rename('cluster'),  self.degree.reindex(self.clustering.index)], 1).to_csv('%s/clust_pos_cluster.txt' % self.res_path,sep='\t')

        
    def __net_analysis_otheromics(self):
        print('Loading The Network...')
        temp=self.network_ori
        temp=temp[(temp['weight']>temp['weight'].quantile(1-self.topx)) | (temp['weight']<temp['weight'].quantile(self.topx))]
        g= ig.Graph.TupleList(zip(temp['level_0'],temp['level_1'],temp['weight']),weights=True)
        print('Cluster Analysis...')
        G_pos = g.subgraph_edges(g.es.select(weight_gt = 0), delete_vertices=False)
        G_neg = g.subgraph_edges(g.es.select(weight_lt = 0), delete_vertices=False)
        G_neg.es['weight'] = [-w for w in G_neg.es['weight']]
        part_pos = leidenalg.ModularityVertexPartition(G_pos, weights='weight')
        part_neg = leidenalg.ModularityVertexPartition(G_neg, weights='weight');
        optimiser = leidenalg.Optimiser()
        diff = optimiser.optimise_partition_multiplex([part_pos, part_neg],layer_weights=[1,-1], n_iterations=-1)
        clust = pd.DataFrame(pd.Series(part_pos.membership+part_neg.membership,index=G_pos.vs['name']+G_neg.vs['name'])).reset_index().drop_duplicates().set_index('index')[0]
        thr=self.cluster_size
        temp_c=clust.value_counts()[clust.value_counts()>thr].index.tolist()
        self.degree_combi = pd.Series(g.degree(), index = g.vs['name'], name = 'degree')
        self.clustering_combi=clust[clust.isin(temp_c)]
        self.modularity_combi=diff
        self.network_combi=g
        
        temp=self.network_ori.merge(pd.DataFrame(self.clustering_combi).reset_index(),how='left',left_on='level_0',right_on='index').merge(pd.DataFrame(self.clustering_combi).reset_index(),how='left',left_on='level_1',right_on='index').dropna()
        temp2=temp[['0_x','0_y']]
        temp2['count'] = 1
        temp2=temp2.groupby(['0_x','0_y']).count().reset_index().pivot_table(index='0_x',columns='0_y',values='count').fillna(0)
        temp2=pd.DataFrame([(i,j,(temp2.loc[i,j]+temp2.loc[j,i]),(temp2.loc[i,j]+temp2.loc[j,i])/(self.clustering_combi.value_counts()[i]+self.clustering_combi.value_counts()[j])) for i in temp2.index for j in temp2.columns if (i < j)],columns=['Source','Target','Weight1','Weight2'])
        temp2.astype(int).to_csv('%s/edge_combi_cluster.txt' % self.res_path,sep='\t')
        pd.DataFrame(self.clustering_combi.rename('Size').value_counts(),columns=['Size']).to_csv('%s/node_combi_cluster.txt' % self.res_path,sep='\t')
        pd.concat([self.clustering_combi.rename('cluster'), self.degree_combi.reindex(self.clustering_combi.index)], 1).to_csv('%s/clust_combi_cluster.txt' % self.res_path,sep='\t')

    
    def save_network(self):
        pickle_out = open('%s/network_object.pkl' % self.res_path,"wb")
        pickle.dump(self, pickle_out)
        pickle_out.close()
        
def impute_knn(data):
    imputer = KNNImputer(n_neighbors=2)
    After_imputation = imputer.fit_transform(data)
    return pd.DataFrame(After_imputation, index = data.index, columns = data.columns)

## Data Loading

In [ ]:
supp1 = pd.ExcelFile('SupplementaryTables/Supplementary Table 1.xlsx') # Metadata, PFT, Ashcroft, WEight, Transcriptomics
supp3 = pd.ExcelFile('SupplementaryTables/Supplementary Table 3.xlsx') # Metabolomics
supp5 = pd.ExcelFile('SupplementaryTables/Supplementary Table 5.xlsx') # Human Transcriptomics
supp6 = pd.ExcelFile('SupplementaryTables/Supplementary Table 6.xlsx') # iDREM
supp7 = pd.ExcelFile('SupplementaryTables/Supplementary Table 7.xlsx') # CB1R

## Figure 1B

Created with GraphPad/Prism

In [ ]:
temp = supp1.parse('Metadata and PFTs', index_col = 0)
metadata = temp.iloc[0:, 0:2]
metadata['Conditions'] = ["%s (%s)" % (metadata['Treatment'][i], metadata['Day'][i]) if metadata['Treatment'][i] != 'Control' else 'Control' for i in metadata.index]
weight = supp1.parse('BodyWeight', index_col = 0)
weight = (weight.T/weight['Day-0']).T*100
weight.columns = [int(i.split('-')[1]) for i in weight.columns]

In [ ]:
temp = weight.unstack().reset_index().dropna()
plt.figure(figsize = (3, 3))
g = sns.pointplot(data = temp, x = 'level_0', y = 0, aspect=5, color = 'grey', errorbar = 'se', errwidth= 1, capsize=.5,scale = 0.2)
xticks = [i for i in weight.columns if i % 7 == 0]
g.set_xticks(xticks)
g.set_xticklabels(xticks, fontsize = 15)#, rotation = 30, ha = 'right')
g.set_yticklabels(['%d' % j if int(j) == j else '%.2f' % j for j in g.get_yticks()], fontsize = 15)
g.set_xlabel('Days\nPost-Bleomycin', fontsize = 15, fontweight = 'bold')
g.set_xlim(0, 29)
g.set_ylabel('')
plt.savefig('Figures/Figure1B.pdf')

## Figure 1C-G

In [ ]:
temp = supp1.parse('Metadata and PFTs', index_col = 0)
metadata = temp.iloc[0:, 0:2]
metadata['Conditions'] = ["%s (%s)" % (metadata['Treatment'][i], metadata['Day'][i]) if metadata['Treatment'][i] != 'Control' else 'Control' for i in metadata.index]
PFT = temp.iloc[0:, 2:]

In [ ]:
fig, axs = plt.subplots(figsize = (10, 6),nrows = 2, ncols = 3, sharex=True)
axs = sum([list(i) for i in axs], [])

for num, pft in enumerate(PFT.columns):
    temp = pd.concat([PFT[pft], metadata],1).reset_index()
    order = ['Control','Bleomycin (7 Days)', 'Bleomycin (14 Days)','Bleomycin (21 Days)', 'Bleomycin (28 Days)']
    box_pairs = [
        ('Control', 'Bleomycin (7 Days)'),
        ('Control', 'Bleomycin (14 Days)'),
        ('Control', 'Bleomycin (21 Days)'),
        ('Control', 'Bleomycin (28 Days)')
                ]
    
    pval_sel = [
        f_oneway(temp[temp['Conditions'] == 'Bleomycin (7 Days)'][pft], 
             temp[temp['Conditions'] == 'Control'][pft])[1],
        f_oneway(temp[temp['Conditions'] == 'Bleomycin (14 Days)'][pft], 
             temp[temp['Conditions'] == 'Control'][pft])[1],
        f_oneway(temp[temp['Conditions'] == 'Bleomycin (21 Days)'][pft], 
             temp[temp['Conditions'] == 'Control'][pft])[1],
        f_oneway(temp[temp['Conditions'] == 'Bleomycin (28 Days)'][pft], 
             temp[temp['Conditions'] == 'Control'][pft])[1],
    ]
    
    sns.boxplot(data = temp, x = 'Conditions', y = pft, order=order, ax = axs[num])
    add_stat_annotation(data = temp, x = 'Conditions', y = pft, order=order, ax = axs[num],
                        box_pairs=box_pairs,
                        perform_stat_test=False, pvalues=pval_sel,
                        test=None, text_format='star', loc='inside', verbose=0, fontsize = 15)
    axs[num].set_xticklabels(axs[num].get_xticklabels(), fontsize = 15, fontweight = 'bold', rotation = 30, ha = 'right')
    axs[num].set_yticklabels(['%d' % j if int(j) == j else '%.2f' % j for j in axs[num].get_yticks()], fontsize = 15)
    axs[num].set_xlabel('')
    axs[num].set_ylabel('')
    axs[num].set_title('%s' % (pft.replace(' (PFT)', '')), fontsize = 15, fontweight = 'bold')
    plt.subplots_adjust(wspace = 0.3)
plt.savefig('Figures/Figure1C.pdf')

## Figure 1H

In [ ]:
ashcroft = supp1.parse('Ashcroft', index_col = 1)

In [ ]:
fig, ax = plt.subplots(figsize = (3, 2.5))
order = ['Control','Bleomycin (7 Days)', 'Bleomycin (14 Days)','Bleomycin (21 Days)', 'Bleomycin (28 Days)']
box_pairs = [
    ('Control', 'Bleomycin (7 Days)'),
    ('Control', 'Bleomycin (14 Days)'),
    ('Control', 'Bleomycin (21 Days)'),
    ('Control', 'Bleomycin (28 Days)')
            ]

pval_sel = [
    f_oneway(ashcroft[ashcroft['Group'] == 'Bleomycin (7 Days)']['Score'], 
         ashcroft[ashcroft['Group'] == 'Control']['Score'])[1],
    f_oneway(ashcroft[ashcroft['Group'] == 'Bleomycin (14 Days)']['Score'], 
         ashcroft[ashcroft['Group'] == 'Control']['Score'])[1],
    f_oneway(ashcroft[ashcroft['Group'] == 'Bleomycin (21 Days)']['Score'], 
         ashcroft[ashcroft['Group'] == 'Control']['Score'])[1],
    f_oneway(ashcroft[ashcroft['Group'] == 'Bleomycin (28 Days)']['Score'], 
         ashcroft[ashcroft['Group'] == 'Control']['Score'])[1],
]

sns.boxplot(data = ashcroft, x = 'Group', y = 'Score', order=order, ax = ax)
# sns.stripplot(data = ashcroft, x = 'Group', y = 'Score', order=order, marker = 's', color = 'black', alpha = 0.25, ax = ax)
add_stat_annotation(data = ashcroft, x = 'Group', y = 'Score', order=order, ax = ax,
                    box_pairs=box_pairs,
                    perform_stat_test=False, pvalues=pval_sel,
                    test=None, text_format='star', loc='inside', verbose=0, fontsize = 15)
ax.set_xticklabels(ax.get_xticklabels(), fontsize = 15, fontweight = 'bold', rotation = 30, ha = 'right')
ax.set_yticklabels(['%d' % j if int(j) == j else '%.2f' % j for j in ax.get_yticks()], fontsize = 15)
ax.set_xlabel('')
ax.set_ylabel('')
ax.set_ylim([0, 8])
ax.set_title('Ashcroft Score', fontsize = 15, fontweight = 'bold')
plt.subplots_adjust(wspace = 0.3)
plt.savefig('Figures/Figure1D.pdf')

## Supplementary Figure 1set_ylim

In [ ]:
temp = supp1.parse('Metadata and PFTs', index_col = 0)
metadata = temp.iloc[0:, 0:2]
metadata['Conditions'] = ["%s (%s)" % (metadata['Treatment'][i], metadata['Day'][i]) if metadata['Treatment'][i] != 'Control' else 'Control' for i in metadata.index]
qpcr = supp1.parse('qPCR_FibrosisMarkers', index_col = 0)

In [ ]:
fig, axs = plt.subplots(figsize = (10, 12),nrows = 2, ncols = 2, sharex=True)
axs = axs.flatten()
for ax,gene in list(zip(axs, qpcr.columns)):
    fc = pd.concat([qpcr[gene], metadata],1).dropna()
    order = ['Control','Bleomycin (7 Days)', 'Bleomycin (14 Days)','Bleomycin (21 Days)', 'Bleomycin (28 Days)']
    box_pairs = [
        ('Control', 'Bleomycin (7 Days)'),
        ('Control', 'Bleomycin (14 Days)'),
        ('Control', 'Bleomycin (21 Days)'),
        ('Control', 'Bleomycin (28 Days)')
                ]
    
    pval_sel = [
        f_oneway(fc[fc['Conditions'] == 'Bleomycin (7 Days)'][gene], 
             fc[fc['Conditions'] == 'Control'][gene])[1],
        f_oneway(fc[fc['Conditions'] == 'Bleomycin (14 Days)'][gene], 
             fc[fc['Conditions'] == 'Control'][gene])[1],
        f_oneway(fc[fc['Conditions'] == 'Bleomycin (21 Days)'][gene], 
             fc[fc['Conditions'] == 'Control'][gene])[1],
        f_oneway(fc[fc['Conditions'] == 'Bleomycin (28 Days)'][gene], 
             fc[fc['Conditions'] == 'Control'][gene])[1],
    ]
    
    sns.barplot(data = fc, x = 'Conditions', y = gene, order=order, capsize=.2, ax = ax)
    sns.stripplot(data = fc, x = 'Conditions', y = gene, order=order, marker = 's', color = 'black', alpha = 0.75, ax = ax)
    add_stat_annotation(data = fc, x = 'Conditions', y = gene, order=order, ax = ax,
                        box_pairs=box_pairs,
                        perform_stat_test=False, pvalues=pval_sel,
                        test=None, text_format='star', loc='inside', verbose=0, fontsize = 15)
    ax.set_title('%s' % gene, fontsize = 15, fontweight = 'bold')
    ax.set_xticklabels(ax.get_xticklabels(), fontsize = 15, fontweight = 'bold', rotation = 30, ha = 'right')
    ax.set_yticklabels(['%d' % j if int(j) == j else '%.1f' % j for j in ax.get_yticks()], fontsize = 15)
    ax.set_xlabel('')
    ax.set_ylabel('')
    plt.tight_layout()
plt.savefig('Figures/Supp1.pdf', transparent = True)

## Figure 2A

Figure 2A was rotated

In [ ]:
temp = supp1.parse('Metadata and PFTs', index_col = 0)
metadata = temp.iloc[0:, 0:2]
metadata['Conditions'] = ["%s (%s)" % (metadata['Treatment'][i], metadata['Day'][i]) if metadata['Treatment'][i] != 'Control' else 'Control' for i in metadata.index]
tpm = supp1.parse('TPM', index_col = 0)[metadata.index]
sel = filter_tpm(tpm, metadata['Conditions'], thr = 5)

In [ ]:
var = 'Conditions'
sklearn_pca = sklearnPCA(n_components=2)
Y_sklearn = sklearn_pca.fit_transform(np.log1p(sel).fillna(0).T)
plt.figure(figsize=(6,5))
coordinate = pd.DataFrame(Y_sklearn, columns = ['x', 'y'], index = sel.columns) * -1 ## -1 is to rotate the PCA, so Control will be on the left, for aesthetic purpose
coordinate = pd.concat([coordinate, metadata],1)
ax = sns.scatterplot(data = coordinate, x = 'x', y = 'y', s = 300, hue = var, palette = 'colorblind')
ax.set_xlabel('PC1 (%.2f%%)' % (sklearn_pca.explained_variance_ratio_[0]*100), fontsize = 20, fontweight = 'bold')
ax.set_ylabel('PC2 (%.2f%%)' % (sklearn_pca.explained_variance_ratio_[1]*100), fontsize = 20, fontweight = 'bold')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.get_legend().set_title('Groups')
plt.setp(ax.get_legend().get_title(), fontsize='15', fontweight = 'bold')
plt.setp(ax.get_legend().get_texts(), fontsize='15')
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
# plt.savefig('Figures/Figure2A.pdf')

## Differential Expression Analysis

In [ ]:
temp = supp1.parse('Metadata and PFTs', index_col = 0)
metadata = temp.iloc[0:, 0:2]
metadata['Conditions'] = ["%s (%s)" % (metadata['Treatment'][i], metadata['Day'][i]) if metadata['Treatment'][i] != 'Control' else 'Control' for i in metadata.index]
count = supp1.parse('Count', index_col = 0)[metadata.index]

In [ ]:
res_path='StatisticalAnalysis/Transcriptomics_TimeSeries/'
k=DESeq_Piano(res_path)
k.DEseq(count,metadata['Conditions'].str.replace('\(', '_').str.replace(')', '').str.replace(' ', ''))
k.loadGSC(KEGG='lib/KEGG_2019_Mouse.gmt')

cond2='Control'
cond1='Bleomycin_7Days'
k.DEseq_Compare(cond1,cond2,save=True)

cond2='Control'
cond1='Bleomycin_14Days'
k.DEseq_Compare(cond1,cond2,save=True)
k.PIANO(cond1,cond2,save=True)

cond2='Control'
cond1='Bleomycin_21Days'
k.DEseq_Compare(cond1,cond2,save=True)

cond2='Control'
cond1='Bleomycin_28Days'
k.DEseq_Compare(cond1,cond2,save=True)

k.summarizeDEseq()
k.summarizePiano()


## Figure 2B

In [ ]:
deseq = pd.ExcelFile('StatisticalAnalysis/Transcriptomics_TimeSeries/DifferentialExpression_ALL.xlsx')

In [ ]:
res = pd.DataFrame()
for i in deseq.sheet_names:
    temp = deseq.parse(i, index_col = 0)
    temp = temp[temp['P-ADJ'] < 0.05]['DIRECTION'].rename(i)
    res = pd.concat([res, temp],1)
    
res = res[[
    'Bleomycin_7Days_Control',
    'Bleomycin_14Days_Control',
    'Bleomycin_21Days_Control',
    'Bleomycin_28Days_Control'
]]

res.shape
temp = res.unstack().reset_index().dropna().groupby(['level_0',0]).count().reset_index()

In [ ]:
plt.figure(figsize = (5,5))
g = sns.barplot(data = temp, x = 'level_0', y = 'level_1', hue = 0, order = res.columns, hue_order = ['UP', 'DOWN'])
g.set_yticklabels([int(i) for i in g.get_yticks()], fontsize = 15)
g.set_xticklabels(g.get_xticklabels(),rotation=30, ha = 'right', fontsize = 15, fontweight = 'bold')
g.set_xlabel('')
g.set_ylabel('# of DEGs', fontsize= 20, fontweight = 'bold')
g.get_legend().set_title('Direction')
plt.setp(g.get_legend().get_title(), fontsize='15', fontweight = 'bold')
plt.setp(g.get_legend().get_texts(), fontsize='15')
plt.tight_layout()
plt.savefig('Figures/Figure2B.pdf')

## Figure 2C-E

In [ ]:
deseq = pd.ExcelFile('StatisticalAnalysis/Transcriptomics_TimeSeries/DifferentialExpression_ALL.xlsx')

res = pd.DataFrame()
for i in deseq.sheet_names:
    temp = deseq.parse(i, index_col = 0)
    temp = temp[temp['P-ADJ'] < 0.05]['DIRECTION'].rename(i)
    res = pd.concat([res, temp],1)
    
res = res[[
    'Bleomycin_7Days_Control',
    'Bleomycin_14Days_Control',
    'Bleomycin_21Days_Control',
    'Bleomycin_28Days_Control'
]]

In [ ]:
temp = supp1.parse('Metadata and PFTs', index_col = 0)
metadata = temp.iloc[0:, 0:2]
metadata['Conditions'] = ["%s (%s)" % (metadata['Treatment'][i], metadata['Day'][i]) if metadata['Treatment'][i] != 'Control' else 'Control' for i in metadata.index]
tpm = supp1.parse('TPM', index_col = 0)[metadata.index]

In [ ]:
selected_genes = res.dropna(thresh = 4)
normalized_tpm = zscore((tpm.reindex(selected_genes.index)).T).T.rename(columns = metadata['Conditions'].to_dict())

row_linkage = linkage(
    distance.pdist(normalized_tpm.fillna(0)), method='average')
clust = pd.Series([i[0] for i in cut_tree(row_linkage, n_clusters = 2)], index = normalized_tpm.index).sort_values()

In [ ]:
vmin = normalized_tpm.min().min()
vmax = normalized_tpm.max().max()
order = ['Control', 'Bleomycin (7 Days)', 'Bleomycin (14 Days)', 'Bleomycin (21 Days)', 'Bleomycin (28 Days)']
g = sns.clustermap(normalized_tpm.reindex(clust.index).fillna(0)[order], figsize = (5,8), yticklabels = 0, xticklabels = 1, center = 0, cmap = 'RdYlBu_r', col_cluster = False, row_cluster = False, row_linkage = row_linkage, vmin = vmin, vmax = vmax)
g.ax_heatmap.vlines(metadata['Conditions'].value_counts()[order].cumsum(), ymin = g.ax_heatmap.get_ylim()[1], ymax = g.ax_heatmap.get_ylim()[0], color = 'k')
g.ax_heatmap.hlines(clust.value_counts().sort_index(ascending = True).cumsum().tolist(), xmin = g.ax_heatmap.get_xlim()[1], xmax = g.ax_heatmap.get_xlim()[0], color = 'k')
g.ax_heatmap.tick_params(right=False, bottom=False)
xticklabels = metadata['Conditions'].value_counts().sort_index()[order]
g.ax_heatmap.set_xticks(xticklabels.cumsum()-((xticklabels/2)))
g.ax_heatmap.set_xticklabels(xticklabels.index.tolist(), fontsize = 15, fontweight = 'bold', rotation = 30, ha = 'right')
g.ax_heatmap.set_title('DEG in ALL comparisons vs Control\n(%d Genes)' % normalized_tpm.shape[0], fontsize = 20, fontweight = 'bold')
g.savefig('Figures/Figure2C.pdf')

In [ ]:
fig, axs = plt.subplots(figsize = (3, 8), nrows = 2, ncols = 1)
color_mapping = dict(zip(clust.value_counts().index,sns.color_palette("tab10").as_hex()))

for i, cl in enumerate(clust.value_counts().index):
    days = {'A': 0, 'B': 7, 'C': 14, 'D': 21, 'E': 28}
    x_interp = np.linspace(days['A'], days['E'], 100)
    sel_col = ['Control', 'Bleomycin (7 Days)', 'Bleomycin (14 Days)','Bleomycin (21 Days)','Bleomycin (28 Days)']
    mapping = dict(zip(sel_col, ['A', 'B', 'C', 'D', 'E']))
    sel_data = normalized_tpm[sel_col].reindex(clust[clust == cl].index).rename(columns = mapping).T.reset_index().groupby('index').mean().T[['A', 'B', 'C', 'D', 'E']].rename(columns = days).dropna()
    y_interp = interp1d(list(days.values()), list(sel_data.values), kind = 'quadratic')(x_interp)
    data_interp = pd.DataFrame(y_interp, columns = x_interp).unstack().reset_index().groupby('level_0').mean().reset_index()
    data_interp[0] = data_interp[0] - data_interp.loc[0,0]
    temp_max = data_interp.copy()[0]
    g = sns.lineplot(data = data_interp, x = 'level_0', y = 0, ax = axs[i], color = color_mapping[cl], linewidth=5, zorder = 15)
    
    if cl == clust.value_counts().index.tolist()[-1]:
        g.set_xlabel('Days', fontweight = 'bold', fontsize = 20)
        g.set_xticks(list(days.values()))
        g.set_xticklabels(list(days.values()), fontweight = 'bold', fontsize = 15)
    else:
        g.set_xlabel('')
        g.set_xticks([])
    temp_ylim = tuple(g.get_ylim())
    g.vlines([7, 14, 21], ymin = g.get_ylim()[0], ymax = g.get_ylim()[1], color = 'lightgray', zorder = 10)
    g.set_ylim(temp_ylim)
    g.set_yticks([])
    g.set_ylabel('')
    g.set_xlim((0,28))
    
plt.savefig('Figures/Figure2C_trend.pdf')

In [ ]:
KEGG = {i.split('\t')[0]:[j for j in i.rstrip('\n').split('\t')[2:] if j != ''] for i in open('lib/KEGG_2019_Mouse.gmt', 'r').readlines()}
exclude_KEGG = ['Hepatitis B', 'Human T-cell leukemia virus 1 infection', 'Regulation of actin cytoskeleton', 'Measles',
           'MicroRNAs in cancer', 'Pathways in cancer', 'Influenza A', 'Oocyte meiosis', 'Progesterone-mediated oocyte maturation',
           'Ribosome biogenesis in eukaryotes', 'Amoebiasis', 'Small cell lung cancer', 'Epstein-Barr virus infection', 'Human papillomavirus infection',
           'Pertussis', 'Staphylococcus aureus infection', 'Proteoglycans in cancer', 'Leishmaniasis', 'Legionellosis', 'Viral carcinogenesis', 
           'Chagas disease (American trypanosomiasis)', 'Toxoplasmosis', 'Malaria', 'Metabolism of xenobiotics by cytochrome P450', 'Chronic myeloid leukemia',
           'Prion diseases', 'Inflammatory bowel disease (IBD)', 'Kaposi sarcoma-associated herpesvirus infection', 'Parkinson disease', 'Hepatitis C', 'Systemic lupus erythematosus',
           'Tuberculosis', 'Hepatitis B', 'Measles', 'Dilated cardiomyopathy (DCM)', 'Hematopoietic cell lineage', 'Adrenergic signaling in cardiomyocytes',
           'Epstein-Barr virus infection', 'Drug metabolism', 'Salivary secretion', 'Thermogenesis', 'Glioma', 'Human cytomegalovirus infection', 'Melanoma', 'Chemical carcinogenesis',
            'African trypanosomiasis', 'Collecting duct acid secretion', 'African trypanosomiasis', 'Vascular smooth muscle contraction', 'Fluid shear stress and atherosclerosis',
            'Rheumatoid arthritis', 'Allograft rejection', 'Long term depression'
          ]

exclude_GO = ['',
             
             ]

color_mapping = dict(zip(sorted(clust.unique()),sns.color_palette("tab10").as_hex()))


fig, axs = plt.subplots(figsize = (3, 10),nrows = len(clust.unique().tolist()), ncols = 1, sharex=False
                        , gridspec_kw={'height_ratios':[25, 6]})

for j, sel_ax in enumerate(axs):
    print()
    gl = [ha.upper() for ha in clust[clust == j].index.tolist()]
    enr = gp.enrichr(gene_list=gl,
             gene_sets=KEGG,
             background = tpm.shape[0],
             outdir='test/enrichr_kegg2',
             cutoff=1,
             verbose=False)
    temp = enr.res2d[~(
        enr.res2d['Term'].isin(exclude_KEGG) |
        (enr.res2d['Term'].str.contains('isease')) |
         (enr.res2d['Term'].str.contains('arcinoma')) |
          (enr.res2d['Term'].str.contains('ancer')) |
        (enr.res2d['Term'].str.contains('thyroid')) |
        (enr.res2d['Term'].str.contains('yndrome')) |
        (enr.res2d['Term'].str.contains('strogen')) |
        (enr.res2d['Term'].str.contains('elanoge')) |
        (enr.res2d['Term'].str.contains('iabet')) |
        (enr.res2d['Term'].str.contains('ysosome')) |
        (enr.res2d['Term'].str.contains('nfection')) |
        (enr.res2d['Term'].str.contains('ntestin')) |
        (enr.res2d['Term'].str.contains('cardi')) |
        (enr.res2d['Term'].str.contains('mmunodef'))
    )].sort_values('Adjusted P-value', ascending = True)#.iloc[0:5]#.iloc[[2,3,4]]['Term'].tolist()
    temp2 = temp[['Term','P-value','Adjusted P-value']]
    temp2 = temp2[temp2['Adjusted P-value'] < 0.01]
    print(temp2.shape)
    temp2['Term'] = temp2['Term'].str.replace('endoplasmic reticulum', 'ER')
    temp2['-Log10(P-Adj)'] = -np.log10(temp2['Adjusted P-value'])
    sns.barplot(data = temp2, x = '-Log10(P-Adj)', y = 'Term', ax = sel_ax, color = color_mapping[j])
    sel_ax.yaxis.tick_right()
    sel_ax.set_yticklabels([i for i in temp2['Term']], fontsize = 15)
#     sel_ax.set_xticks([])
    sel_ax.set_ylabel('DEG-%d (%d Genes)' % (j, len(gl)), fontsize = 15, fontweight = 'bold')
    sel_ax.tick_params(right = False)
    if j == clust.max():
        sns.despine(ax = sel_ax)
        sel_ax.set_xlabel('-Log10(P-Adj)', fontsize = 15, fontweight = 'bold')
    else:
        sns.despine(ax = sel_ax, bottom = False)
        sel_ax.set_xlabel('')
plt.savefig('Figures/Figure2D-E.pdf')

## Gene Co-Expression Network Generation and Analysis

This analysis might take a long time, especially for the generation of the network. Pre-analyzed networks can be found under "Networks/ReadyObjects"

In [ ]:
temp = supp1.parse('Metadata and PFTs', index_col = 0)
metadata = temp.iloc[0:, 0:2]
metadata['Conditions'] = ["%s (%s)" % (metadata['Treatment'][i], metadata['Day'][i]) if metadata['Treatment'][i] != 'Control' else 'Control' for i in metadata.index]
PFT = temp.iloc[0:, 2:]

tpm = supp1.parse('TPM', index_col = 0)[metadata.index]

In [ ]:
temp = pd.concat([tpm.T, metadata['Conditions']], 1).groupby('Conditions').mean()
temp = temp.T[temp.std() > temp.std().quantile(.25)].index.tolist()
tpm_filt = tpm.reindex(temp)

In [ ]:
coexp = calc(tpm_filt, clin = PFT, pval_thr = 0.05)
coexp.to_csv('Networks/GCN.txt', sep = '\t', index = False)

In [ ]:
topx = 0.25
k=Network_Analysis(filename='Networks/GCN.txt', network_type='transcriptomics',respath='Networks/GCN/',
                   topx=topx, cluster_size = 30)
k.save_network()

### Loading Pre-analyzed networks

In [ ]:
k = pickle.load(open('Networks/ReadyObjects/GCN.pkl', 'rb'))

# iGraph object is under k.network
# Pandas object of the original network (not filtered based on the correlation threshold) is under k.network_ori
# Subnetwork info iss under k.clustering
# Degree centrality is under k.degree

## Figure 3A

Generated with cytoscape using the results of network analysis under "Networks/GCN/" or Supplementary Table 2. The size of the nodes was proportional to the average transitivity calculated below

In [ ]:
x = []
for cl in sorted(k.clustering.unique()):
    k_temp = k.network.subgraph(k.clustering[k.clustering == cl].index)
    print(cl)
    print(k_temp.transitivity_avglocal_undirected())
    x.append([cl, k_temp.transitivity_avglocal_undirected(), cl])

## Figure 3B

In [ ]:
PFT_Degree = k.degree[k.degree.index.str.contains('PFT')].sort_values(ascending = False).reset_index()

sns.barplot(data = PFT_Degree, x = 'degree', y = 'index')
plt.savefig('Figures/Figure3B.pdf')

## Figure 3C

In [ ]:
k = pickle.load(open('Networks/ReadyObjects/GCN.pkl', 'rb'))

clusters = pd.concat([k.clustering.rename('cluster'), k.degree],1)
clusters = clusters[clusters['cluster'].notna()]
clusters['norm_deg'] = ((clusters['degree'] - clusters['degree'].min())/(clusters['degree'].max() - clusters['degree'].min()))

In [ ]:
temp = supp1.parse('Metadata and PFTs', index_col = 0)
metadata = temp.iloc[0:, 0:2]
metadata['Conditions'] = ["%s (%s)" % (metadata['Treatment'][i], metadata['Day'][i]) if metadata['Treatment'][i] != 'Control' else 'Control' for i in metadata.index]
PFT = temp.iloc[0:, 2:]

tpm = supp1.parse('TPM', index_col = 0)[metadata.index]

norm_tpm = zscore(tpm.T).T.rename(columns = metadata['Conditions'].to_dict())
norm_pft = zscore(PFT).T.rename(columns = metadata['Conditions'].to_dict())

In [ ]:
x = []
for cl in sorted(clusters['cluster'].unique()):
    sel_data = norm_tpm.reindex(clusters[clusters['cluster'] == cl].index).rename(columns = metadata['Conditions'].to_dict())#.T#.reset_index().groupby('index').mean().T.rename(columns = days)
    row_linkage = linkage(distance.pdist(sel_data.fillna(0)), method='average')
    dn = dendrogram(row_linkage, no_plot = True)['leaves']
    temp = sel_data.iloc[dn].index.tolist()
    x.append(temp)

In [ ]:
KEGG = {i.split('\t')[0]:[j for j in i.rstrip('\n').split('\t')[2:] if j != ''] for i in open('lib/KEGG_2019_Mouse.gmt', 'r').readlines()}
exclude_KEGG = ['Hepatitis B', 'Human T-cell leukemia virus 1 infection', 'Regulation of actin cytoskeleton', 'Measles',
           'MicroRNAs in cancer', 'Pathways in cancer', 'Influenza A', 'Oocyte meiosis', 'Progesterone-mediated oocyte maturation',
           'Ribosome biogenesis in eukaryotes', 'Amoebiasis', 'Small cell lung cancer', 'Epstein-Barr virus infection', 'Human papillomavirus infection',
           'Pertussis', 'Staphylococcus aureus infection', 'Proteoglycans in cancer', 'Leishmaniasis', 'Legionellosis', 'Viral carcinogenesis', 
           'Chagas disease (American trypanosomiasis)', 'Toxoplasmosis', 'Malaria', 'Metabolism of xenobiotics by cytochrome P450', 'Chronic myeloid leukemia',
           'Prion diseases', 'Inflammatory bowel disease (IBD)', 'Kaposi sarcoma-associated herpesvirus infection', 'Parkinson disease', 'Hepatitis C', 'Systemic lupus erythematosus',
           'Tuberculosis', 'Hepatitis B', 'Measles', 'Dilated cardiomyopathy (DCM)', 'Hematopoietic cell lineage', 'Adrenergic signaling in cardiomyocytes',
           'Epstein-Barr virus infection', 'Drug metabolism', 'Salivary secretion', 'Thermogenesis', 'Glioma', 'Human cytomegalovirus infection', 'Melanoma', 'Chemical carcinogenesis',
            'African trypanosomiasis', 'Collecting duct acid secretion', 'African trypanosomiasis', 'Vascular smooth muscle contraction', 'Fluid shear stress and atherosclerosis',
            'Rheumatoid arthritis', 'Allograft rejection', 'Long term depression'
          ]


In [ ]:
metadata['Conditions']

In [ ]:
days = {'Control': 0, 'Bleomycin (7 Days)': 7, 'Bleomycin (14 Days)': 14, 'Bleomycin (21 Days)': 21, 'Bleomycin (28 Days)': 28}
x_interp = np.linspace(days['Control'], days['Bleomycin (28 Days)'], 100)
color_mapping = dict(zip(sorted(clusters['cluster'].unique()),sns.color_palette("tab10").as_hex()[::-1]))

cluster_color = pd.Series(clusters['cluster']).map(color_mapping)
sel_data = norm_tpm.reindex(sum(x,[]))
cm = sns.clustermap(sel_data.fillna(0), figsize=(12.63, 10),  yticklabels = 0, xticklabels = 0, center = 0, cmap = 'RdYlBu_r', row_colors = cluster_color, row_cluster = False, col_cluster = False)
cm.cax.set_visible(False)
cm.gs.update(left=0.05, right=0.25)
cm.ax_heatmap.vlines(metadata.groupby('Conditions').count()['Treatment'].cumsum().tolist(), ymin = 0, ymax = sel_data.shape[0], color = 'gray')
cm.ax_heatmap.hlines(np.cumsum([len(i) for i in x]), xmin = 0, xmax = sel_data.shape[1], color = 'black')
cm.ax_heatmap.set_xlabel('Days', fontweight = 'bold', fontsize = 20)
cm.ax_heatmap.set_ylabel('', fontweight = 'bold', fontsize = 20)
cm.ax_heatmap.set_xticks((metadata.groupby('Conditions').count().cumsum() - metadata.groupby('Conditions').count()/2)['Treatment'].tolist())
cm.ax_heatmap.set_xticklabels([0, 7, 14, 21, 28], fontweight = 'bold', fontsize = 15)

gs = GridSpec(int(clusters['cluster'].max()+2),5, top = .95, left = 0.26, hspace = 0.2, 
              wspace = 0.21,width_ratios=[.5,0.1, .5,  1, 0.25])

for cl in sorted(clusters['cluster'].unique().astype(int)):
    sel_ax = cm.fig.add_subplot(gs[cl+1, 0])
    sel_data = norm_tpm.reindex(
        clusters[clusters['cluster'] == cl].index
    ).rename(columns = metadata['Conditions'].to_dict()).T.reset_index().groupby('index').mean().T.rename(columns = days).dropna()[days.values()]
    y_interp = interp1d(list(days.values()), list(sel_data.values), kind = 'quadratic')(x_interp)
    data_interp = pd.DataFrame(y_interp, columns = x_interp).unstack().reset_index().groupby('level_0').mean().reset_index()
    temp_max = data_interp.copy()[0]
    g = sns.lineplot(data = data_interp, x = 'level_0', y = 0, ax = sel_ax, color = color_mapping[cl])
    sel_data = norm_pft.reindex(
        clusters[clusters['cluster'] == cl].index
    ).rename(columns = metadata['Conditions'].to_dict()).T.reset_index().groupby('index').mean().T.rename(columns = days).dropna()[days.values()]
    
    if sel_data.shape[0] > 0:
        y_interp = interp1d(list(days.values()), list(sel_data.values), kind = 'quadratic')(x_interp)
        temp_max = pd.concat([temp_max, data_interp.copy()[0]])
        data_interp = pd.DataFrame(y_interp, columns = x_interp).unstack().reset_index().groupby('level_0').mean().reset_index()
        g = sns.lineplot(data = data_interp, x = 'level_0', y = 0, ax = sel_ax, color = color_mapping[cl], linestyle='--')
    if cl == clusters['cluster'].max():
        g.set_xlabel('Days', fontweight = 'bold', fontsize = 20)
        g.set_xticks(list(days.values()))
        g.set_xticklabels(list(days.values()), fontweight = 'bold', fontsize = 15)
    else:
        g.set_xlabel('')
        g.set_xticks([])
    temp_ylim = tuple(g.get_ylim())
    g.vlines([7, 14, 21], ymin = g.get_ylim()[0], ymax = g.get_ylim()[1], color = 'lightgray')
    g.set_ylim(temp_ylim)
    g.set_yticks([])
    g.set_ylabel('')
    g.set_xlim((0,28))

sel_term = []
for j,i in enumerate(x):
    sel_ax = cm.fig.add_subplot(gs[j+1, 2])
    enr = gp.enrichr(gene_list=[ha.upper() for ha in i],
             gene_sets=KEGG,
             background = tpm.shape[0],
             outdir='test/enrichr_kegg2',
             cutoff=0.5,
             verbose=False)
    temp = enr.res2d[~(
        enr.res2d['Term'].isin(exclude_KEGG) |
        (enr.res2d['Term'].str.contains('isease')) |
         (enr.res2d['Term'].str.contains('arcinoma')) |
          (enr.res2d['Term'].str.contains('ancer')) |
        (enr.res2d['Term'].str.contains('thyroid')) |
        (enr.res2d['Term'].str.contains('yndrome')) |
        (enr.res2d['Term'].str.contains('strogen')) |
        (enr.res2d['Term'].str.contains('elanoge')) |
        (enr.res2d['Term'].str.contains('iabet')) |
        (enr.res2d['Term'].str.contains('ysosome')) |
        (enr.res2d['Term'].str.contains('nfection')) |
        (enr.res2d['Term'].str.contains('homolog')) |
        (enr.res2d['Term'].str.contains('Proteasome')) |
        (enr.res2d['Term'].str.contains('ntestin')) |
        (enr.res2d['Term'].str.contains('cardi')) |
        (enr.res2d['Term'].str.contains('Cardi')) |
        (enr.res2d['Term'].str.contains('nemia')) |
        (enr.res2d['Term'].str.contains('mmunodef'))
    )]
    temp = temp.sort_values('P-value', ascending = True).iloc[0:5]
    temp2 = temp[['Term','Adjusted P-value']]
    sel_term.append(temp2['Term'].tolist())

    temp2['Term'] = temp2['Term'].str.replace('endoplasmic reticulum', 'ER')
    temp2['-Log10(P-Adj)'] = -np.log10(temp2['Adjusted P-value'])
    sns.barplot(data = temp2, x = '-Log10(P-Adj)', y = 'Term', ax = sel_ax, color = color_mapping[j])
    sel_ax.yaxis.tick_right()
    sel_ax.set_yticklabels([i for i in temp['Term']], fontsize = 15)
    sel_ax.set_xticks([0])
    if j == len(x) - 1:
        sel_ax.set_xlabel('-Log10(P-Adj)', fontweight = 'bold', fontsize = 20)
        sel_ax.set_xticks([0])
        sel_ax.set_xticklabels([], fontweight = 'bold', fontsize = 15)
        sns.despine(ax = sel_ax, left = False, bottom = False, right = True, top = True)
    else:
        sel_ax.set_xlabel('')
        sel_ax.set_xticks([])
        sns.despine(ax = sel_ax, left = False, bottom = True, right = True, top = True)
    sel_ax.set_ylabel('')
    sel_ax.tick_params(right=False, bottom=False)
    
for cl in sorted(clusters['cluster'].unique().astype(int)):
    sel_ax = cm.fig.add_subplot(gs[cl+1, 4])
    topx = 5
    temp = pd.DataFrame()
    for j in sel_term[cl]:
        gn = [i.capitalize() for i in KEGG[j]]
        temp = pd.concat([temp, clusters[clusters['cluster'] == cl].reindex(gn).dropna().sort_values('degree', ascending = False).iloc[0:1].reset_index()])
#     temp = clusters[clusters['cluster'] == cl].reindex(sel_term[cl]).dropna().sort_values('degree', ascending = False).iloc[0:topx].reset_index()
    sns.scatterplot(data = temp, x = 'degree', y = 'index', ax = sel_ax, color = color_mapping[cl], size = 'norm_deg', legend = False)
    sel_ax.yaxis.tick_right()
    sel_ax.set_yticklabels([i for i in temp['index']], fontsize = 15)
    sel_ax.set_xticks([0])
    if cl == clusters['cluster'].max():
        sel_ax.set_xlabel('Top Related Nodes\n(Degree)', fontweight = 'bold', fontsize = 20)
        sel_ax.set_xticks([0])
        sel_ax.set_xticklabels([], fontweight = 'bold', fontsize = 15)
        sns.despine(ax = sel_ax, left = False, bottom = False, right = True, top = True)
    else:
        sel_ax.set_xlabel('')
        sel_ax.set_xticks([])
        sns.despine(ax = sel_ax, left = False, bottom = True, right = True, top = True)
    sel_ax.set_ylabel('')
    sel_ax.tick_params(right=False, bottom=False)

plt.savefig('Figures/Figure3C.pdf')

## Figure 3D

In [ ]:
pft1 = ['Stiffness Index (PFT)', 'Peripheral Airway Resistance (PFT)', 'Central Airway Resistance (PFT)']
pft2 = ['Forced Expiratory Volume (PFT)', 'Forced Vital Capacity (PFT)', 'Inspiratory Capacity (PFT) ']

In [ ]:
topx = 0.25
final_edges = k.network_ori[k.network_ori['weight']>k.network_ori['weight'].quantile(1-topx)]

In [ ]:
net_pft2

In [ ]:
net_pft1 = final_edges[final_edges['level_0'].isin(pft1) | final_edges['level_1'].isin(pft1)].sort_values('weight', ascending = False)
net_pft2 = final_edges[final_edges['level_0'].isin(pft2) | final_edges['level_1'].isin(pft2)].sort_values('weight', ascending = False)
net_pft1.to_csv('Networks/GCN/net_pft1.txt', sep = '\t', index = False)
net_pft2.to_csv('Networks/GCN/net_pft2.txt', sep = '\t', index = False)

## Figure 4A

In [ ]:
temp = supp1.parse('Metadata and PFTs', index_col = 0)
metadata = temp.iloc[0:, 0:2]
metadata['Conditions'] = ["%s (%s)" % (metadata['Treatment'][i], metadata['Day'][i]) if metadata['Treatment'][i] != 'Control' else 'Control' for i in metadata.index]

data = supp3.parse('Putative Metabolomics', index_col = 0)[metadata.index]

mapping_putative = supp3.parse('Putative Mapping', index_col = 0)

## Remove data that were detected in less than 2 samples per group
remove = data[(pd.concat([data.T.notna(),metadata['Conditions']],1).groupby('Conditions').sum() < 2).sum() > 0].index.tolist()
removed_mets = mapping_putative.reindex(remove)
data_putative = data[~data.index.isin(remove)]

Control = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Control'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))
Bleo_7 = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Bleomycin (7 Days)'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))
Bleo_14 = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Bleomycin (14 Days)'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))
Bleo_21 = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Bleomycin (21 Days)'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))
Bleo_28 = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Bleomycin (28 Days)'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))


data_putative_imputed = pd.concat([Control, Bleo_7, Bleo_14, Bleo_21, Bleo_28]).T.dropna()

norm_data_putative_imputed = zscore(data_putative_imputed.T).T

In [ ]:
var = 'Conditions'
sklearn_pca = sklearnPCA(n_components=2)
Y_sklearn = sklearn_pca.fit_transform((norm_data_putative_imputed).T)
plt.figure(figsize=(6,5))
coordinate = -1*pd.DataFrame(Y_sklearn, columns = ['x', 'y'], index = norm_data_putative_imputed.columns) ## -1 is to switch 7 days to left.
coordinate = pd.concat([coordinate, metadata],1)
ax = sns.scatterplot(data = coordinate, x = 'x', y = 'y', s = 300, hue = var, palette = 'colorblind')
ax.set_xlabel('PC1 (%.2f%%)' % (sklearn_pca.explained_variance_ratio_[0]*100), fontsize = 20, fontweight = 'bold')
ax.set_ylabel('PC2 (%.2f%%)' % (sklearn_pca.explained_variance_ratio_[1]*100), fontsize = 20, fontweight = 'bold')
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.get_legend().set_title('Groups')
plt.setp(ax.get_legend().get_title(), fontsize='15', fontweight = 'bold')
plt.setp(ax.get_legend().get_texts(), fontsize='15')
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.savefig('Figures/Figure4A.pdf')

## Statistical Analysis of Metabolomics

In [ ]:
temp = supp1.parse('Metadata and PFTs', index_col = 0)
metadata = temp.iloc[0:, 0:2]
metadata['Conditions'] = ["%s (%s)" % (metadata['Treatment'][i], metadata['Day'][i]) if metadata['Treatment'][i] != 'Control' else 'Control' for i in metadata.index]

data = supp3.parse('Putative Metabolomics', index_col = 0)[metadata.index]

mapping_putative = supp3.parse('Putative Mapping', index_col = 0)

## Remove data that were detected in less than 2 samples per group
remove = data[(pd.concat([data.T.notna(),metadata['Conditions']],1).groupby('Conditions').sum() < 2).sum() > 0].index.tolist()
removed_mets = mapping_putative.reindex(remove)
data_putative = data[~data.index.isin(remove)]

In [ ]:
Control = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Control'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))
Bleo_7 = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Bleomycin (7 Days)'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))
Bleo_14 = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Bleomycin (14 Days)'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))
Bleo_21 = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Bleomycin (21 Days)'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))
Bleo_28 = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Bleomycin (28 Days)'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))

res = []

for i in data_putative.index:
    temp1 = ttest_ind(Bleo_7[i].dropna(), Control[i].dropna(), nan_policy='omit')
    temp2 = ttest_ind(Bleo_14[i].dropna(), Control[i].dropna(), nan_policy='omit')
    temp3 = ttest_ind(Bleo_21[i].dropna(), Control[i].dropna(), nan_policy='omit')
    temp4 = ttest_ind(Bleo_28[i].dropna(), Control[i].dropna(), nan_policy='omit')
    res.append([
                i, 
                np.log(Bleo_7[i].mean()/Control[i].mean()), temp1[1],
                np.log(Bleo_14[i].mean()/Control[i].mean()), temp2[1],
                np.log(Bleo_21[i].mean()/Control[i].mean()), temp3[1],
                np.log(Bleo_28[i].mean()/Control[i].mean()), temp4[1]
               ])
res = pd.DataFrame(res,columns=['Metabolite',
                                'Log2FoldChange (Bleo (7 Days) vs Control)', 'P-value (Bleo (7 Days) vs Control)',
                                'Log2FoldChange (Bleo (14 Days) vs Control)', 'P-value (Bleo (14 Days) vs Control)',
                                'Log2FoldChange (Bleo (21 Days) vs Control)', 'P-value (Bleo (21 Days) vs Control)',
                                'Log2FoldChange (Bleo (28 Days) vs Control)', 'P-value (Bleo (28 Days) vs Control)'
                                ]).set_index('Metabolite')

res['P-Adj (Bleo (7 Days) vs Control)'] = multipletests(res['P-value (Bleo (7 Days) vs Control)'].fillna(1),method='fdr_bh')[1]
res['P-Adj (Bleo (14 Days) vs Control)'] = multipletests(res['P-value (Bleo (14 Days) vs Control)'].fillna(1),method='fdr_bh')[1]
res['P-Adj (Bleo (21 Days) vs Control)'] = multipletests(res['P-value (Bleo (21 Days) vs Control)'].fillna(1),method='fdr_bh')[1]
res['P-Adj (Bleo (28 Days) vs Control)'] = multipletests(res['P-value (Bleo (28 Days) vs Control)'].fillna(1),method='fdr_bh')[1]
res=pd.concat([mapping_putative.reindex(res.index),res],1)

res['DIRECTION (Bleo (7 Days) vs Control)'] = ['UP' if i > 0 else 'DOWN' for i in res['Log2FoldChange (Bleo (7 Days) vs Control)']]
res['DIRECTION (Bleo (14 Days) vs Control)'] = ['UP' if i > 0 else 'DOWN' for i in res['Log2FoldChange (Bleo (14 Days) vs Control)']]
res['DIRECTION (Bleo (21 Days) vs Control)'] = ['UP' if i > 0 else 'DOWN' for i in res['Log2FoldChange (Bleo (21 Days) vs Control)']]
res['DIRECTION (Bleo (28 Days) vs Control)'] = ['UP' if i > 0 else 'DOWN' for i in res['Log2FoldChange (Bleo (28 Days) vs Control)']]

res = res[mapping_putative.columns.tolist()+ [
    'Log2FoldChange (Bleo (7 Days) vs Control)', 'P-value (Bleo (7 Days) vs Control)','P-Adj (Bleo (7 Days) vs Control)', 'DIRECTION (Bleo (7 Days) vs Control)',
    'Log2FoldChange (Bleo (14 Days) vs Control)', 'P-value (Bleo (14 Days) vs Control)','P-Adj (Bleo (14 Days) vs Control)', 'DIRECTION (Bleo (14 Days) vs Control)',
    'Log2FoldChange (Bleo (21 Days) vs Control)', 'P-value (Bleo (21 Days) vs Control)','P-Adj (Bleo (21 Days) vs Control)','DIRECTION (Bleo (21 Days) vs Control)',
    'Log2FoldChange (Bleo (28 Days) vs Control)', 'P-value (Bleo (28 Days) vs Control)','P-Adj (Bleo (28 Days) vs Control)','DIRECTION (Bleo (28 Days) vs Control)'
]]

res.to_csv('StatisticalAnalysis/Metabolomics/Results.txt', sep = '\t')

## Figure 4B

In [ ]:
res = pd.read_csv('StatisticalAnalysis/Metabolomics/Results.txt', sep = '\t', index_col = 0)

In [ ]:
res_padj = res[['P-Adj (Bleo (7 Days) vs Control)', 'P-Adj (Bleo (14 Days) vs Control)', 'P-Adj (Bleo (21 Days) vs Control)', 'P-Adj (Bleo (28 Days) vs Control)']]
res_padj.columns = res_padj.columns.str.replace('P-Adj', '')
res = res[['DIRECTION (Bleo (7 Days) vs Control)', 'DIRECTION (Bleo (14 Days) vs Control)', 'DIRECTION (Bleo (21 Days) vs Control)', 'DIRECTION (Bleo (28 Days) vs Control)']]
res.columns = res.columns.str.replace('DIRECTION', '')
res = res[res_padj < 0.05]

In [ ]:
temp = res.unstack().reset_index().dropna().groupby(['level_0',0]).count().reset_index()

In [ ]:
plt.figure(figsize = (5,5))
g = sns.barplot(data = temp, x = 'level_0', y = 'Metabolite', hue = 0, order = res.columns, hue_order = ['UP', 'DOWN'])
g.set_yticklabels([int(i) for i in g.get_yticks()], fontsize = 15)
g.set_xticklabels(g.get_xticklabels(),rotation=30, ha = 'right', fontsize = 15, fontweight = 'bold')
g.set_xlabel('')
g.set_ylabel('# of DM', fontsize= 20, fontweight = 'bold')
g.get_legend().set_title('Direction')
plt.setp(g.get_legend().get_title(), fontsize='15', fontweight = 'bold')
plt.setp(g.get_legend().get_texts(), fontsize='15')
plt.tight_layout()
plt.savefig('Figures/Figure4B.pdf')

## Figure 4C

In [ ]:
res = pd.read_csv('StatisticalAnalysis/Metabolomics/Results.txt', sep = '\t', index_col = 0)
res = res[['P-Adj (Bleo (7 Days) vs Control)', 'P-Adj (Bleo (14 Days) vs Control)', 'P-Adj (Bleo (21 Days) vs Control)', 'P-Adj (Bleo (28 Days) vs Control)']]
res = res[res < 0.05].dropna(thresh = 2)

In [ ]:
temp = supp1.parse('Metadata and PFTs', index_col = 0)
metadata = temp.iloc[0:, 0:2]
metadata['Conditions'] = ["%s (%s)" % (metadata['Treatment'][i], metadata['Day'][i]) if metadata['Treatment'][i] != 'Control' else 'Control' for i in metadata.index]

data = supp3.parse('Putative Metabolomics', index_col = 0)[metadata.index]

mapping_putative = supp3.parse('Putative Mapping', index_col = 0)

## Remove data that were detected in less than 2 samples per group
remove = data[(pd.concat([data.T.notna(),metadata['Conditions']],1).groupby('Conditions').sum() < 2).sum() > 0].index.tolist()
removed_mets = mapping_putative.reindex(remove)
data_putative = data[~data.index.isin(remove)]

Control = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Control'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))
Bleo_7 = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Bleomycin (7 Days)'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))
Bleo_14 = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Bleomycin (14 Days)'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))
Bleo_21 = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Bleomycin (21 Days)'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))
Bleo_28 = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Bleomycin (28 Days)'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))


data_putative_imputed = pd.concat([Control, Bleo_7, Bleo_14, Bleo_21, Bleo_28]).T.dropna()

norm_data_putative_imputed = zscore(data_putative_imputed.T).T.reindex(res.index).rename(columns = metadata['Conditions'])

In [ ]:
row_linkage = linkage(
    distance.pdist(norm_data_putative_imputed.fillna(0)), method='average')
clust = pd.Series([i[0] for i in cut_tree(row_linkage, n_clusters = 2)], 
                  index = norm_data_putative_imputed.index).sort_values().replace({0:1, 1:0}) ## Just to put the up-regulated clusters as DM-0

clust.value_counts()

In [ ]:
vmin = norm_data_putative_imputed.min().min()
vmax = norm_data_putative_imputed.max().max()
order = ['Control', 'Bleomycin (7 Days)', 'Bleomycin (14 Days)', 'Bleomycin (21 Days)', 'Bleomycin (28 Days)']
cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#0000a5",'#0000d8',"#FFFAF0",'#d80000',"#a50000"])
g = sns.clustermap(norm_data_putative_imputed.reindex(clust.index[::-1]).fillna(0)[order], figsize = (5,8), xticklabels = 1, yticklabels = 0, center = 0, cmap = 'RdYlBu_r', col_cluster = False, row_cluster = False, row_linkage = row_linkage, vmin = vmin, vmax = vmax)
g.ax_heatmap.vlines(metadata['Conditions'].value_counts()[order].cumsum(), ymin = g.ax_heatmap.get_ylim()[1], ymax = g.ax_heatmap.get_ylim()[0], color = 'k')
g.ax_heatmap.hlines(clust.value_counts().sort_index(ascending = False)[::-1].cumsum().tolist(), xmin = g.ax_heatmap.get_xlim()[1], xmax = g.ax_heatmap.get_xlim()[0], color = 'k')
g.ax_heatmap.tick_params(right=False, bottom=False)
xticklabels = metadata['Conditions'].value_counts().sort_index()[order]
g.ax_heatmap.set_xticks(xticklabels.cumsum()-((xticklabels/2)))
g.ax_heatmap.set_xticklabels(xticklabels.index.tolist(), fontsize = 15, fontweight = 'bold', rotation = 30, ha = 'right')
g.savefig('Figures/Figure4C.pdf')

In [ ]:
fig, axs = plt.subplots(figsize = (3, 8), nrows = 2, ncols = 1)
color_mapping = dict(zip(clust.value_counts().index,sns.color_palette("tab10").as_hex()))

for i, cl in enumerate(clust.value_counts().index):
    days = {'A': 0, 'B': 7, 'C': 14, 'D': 21, 'E': 28}
    x_interp = np.linspace(days['A'], days['E'], 100)
    sel_col = ['Control', 'Bleomycin (7 Days)', 'Bleomycin (14 Days)', 'Bleomycin (21 Days)', 'Bleomycin (28 Days)']
    mapping = dict(zip(sel_col, ['A', 'B', 'C', 'D', 'E']))
    sel_data = norm_data_putative_imputed[sel_col].reindex(clust[clust == cl].index).rename(columns = mapping).T.reset_index().groupby('index').mean().T[['A', 'B', 'C', 'D', 'E']].rename(columns = days).dropna()
    y_interp = interp1d(list(days.values()), list(sel_data.values), kind = 'quadratic')(x_interp)
    data_interp = pd.DataFrame(y_interp, columns = x_interp).unstack().reset_index().groupby('level_0').mean().reset_index()
    data_interp[0] = data_interp[0] - data_interp.loc[0,0]
    temp_max = data_interp.copy()[0]
    g = sns.lineplot(data = data_interp, x = 'level_0', y = 0, ax = axs[i], color = color_mapping[cl], linewidth=5, zorder = 15)
    
    if cl == clust.value_counts().index.tolist()[-1]:
        g.set_xlabel('Days', fontweight = 'bold', fontsize = 20)
        g.set_xticks(list(days.values()))
        g.set_xticklabels(list(days.values()), fontweight = 'bold', fontsize = 15)
    else:
        g.set_xlabel('')
        g.set_xticks([])
    temp_ylim = tuple(g.get_ylim())
    g.vlines([7, 14, 21], ymin = g.get_ylim()[0], ymax = g.get_ylim()[1], color = 'lightgray', zorder = 10)
    g.set_ylim(temp_ylim)
    
    g.set_yticks([])
    g.set_ylabel('')
    g.set_xlim((0,28))
    
plt.savefig('Figures/Figure4C_trend.pdf')

## Figure 4D-E

Do figure 5C first, because the metabolite enrichment will use the DM-0 and DM-1 assignment calculated there

In [ ]:
pathways = supp3.parse('Pathway Mapping').dropna()
pathways = pathways[pathways['ID'] != '-']
pathways = pathways.groupby('Pathway')['ID'].apply(lambda x: list(x.unique())).to_dict()
pw_bg = mapping_putative.shape[0]

In [ ]:
cl = 1
temp = mapping_putative.reindex(clust[clust == cl].index)['HMDB ID'].dropna()
met_list = []
for i in temp.tolist():
    if i.count('HMDB') == 0:
        pass
    elif i.count('HMDB') > 1:
        for j in i.replace(',','\n').split('\n'):
            met_list.append(j)
    else:
        met_list.append(i.replace('\n',''))

In [ ]:
fig, axs = plt.subplots(figsize = (3, 8), nrows = 2, ncols = 1, sharex = False)
color_mapping = dict(zip(clust.value_counts().index,sns.color_palette("tab10").as_hex()))

for cl in clust.unique()[::-1]:
    print(cl)
    met_list = clust[clust == cl].index.tolist()
    x = []
    for pw in list(pathways.keys()):
        hist_mets = mapping_putative.reindex(clust.reindex(list(set(met_list).intersection(set(pathways[pw])))).index)['Compound name']
        hits = len(hist_mets)
        num_ofpw = len(set(sum(list(pathways.values()), [])))
        num_ofmets_inpw = len(set(pathways[pw]))
        total_mets = len(set(met_list))
        pval = 1-hypergeom.cdf(hits,num_ofpw-num_ofmets_inpw,total_mets,num_ofmets_inpw)
        x.append([pw, hits, num_ofmets_inpw, ';'.join(list(hist_mets)), pval])
    x = pd.DataFrame(x, columns = ['Pathways', '# of Hits', 'Total Mets', 'Hits Metabolites', 'pval']).set_index('Pathways')
    x = x[(x['# of Hits'] > 0) & (x['Total Mets'] > 1)]
    x = x[~x.index.isin(['Prostate cancer'])]
    print(x[x['pval'] < 0.05].shape[0])
    temp = x[x['pval'] < 0.05].sort_values('pval').iloc[0:10].reset_index()
    temp['-Log10(P)'] = -np.log10(temp['pval'] + 1E-2)
    temp['color'] = '#a50000'
    g = sns.barplot(data = temp, x = '-Log10(P)', y = 'Pathways', color = color_mapping[cl], ax = axs[cl])
    
    g.set_ylabel('')
    sns.despine()
    g.set_yticklabels(g.get_yticklabels(), fontsize = 15)
    if cl == 0:
        g.set_xlabel('')
    else:
        g.set_xlabel('-Log10(P)', fontsize = 20, fontweight = 'bold')
    g.set_xticklabels(['%.1f' % i for i in g.get_xticks()], fontsize = 15)
    g.set_title('DM-%d' % cl, fontsize = 20, fontweight = 'bold')
plt.savefig('Figures/Figure4D-E.pdf')

### Loading Pre-analyzed networks

In [ ]:
k = pickle.load(open('Networks/ReadyObjects/MCN.pkl', 'rb'))

# iGraph object is under k.network
# Pandas object of the original network (not filtered based on the correlation threshold) is under k.network_ori
# Subnetwork info iss under k.clustering
# Degree centrality is under k.degree

## Figure 5A

In [ ]:
x = []
for cl in sorted(k.clustering.unique()):
    k_temp = k.network.subgraph(k.clustering[k.clustering == cl].index)
    print(cl)
    print(k_temp.transitivity_avglocal_undirected())
    x.append([cl, k_temp.transitivity_avglocal_undirected(), cl])

## Figure 5B

In [ ]:
k = pickle.load(open('Networks/ReadyObjects/MCN.pkl', 'rb'))
mapping_putative = supp3.parse('Putative Mapping', index_col = 0)

clusters = pd.concat([k.clustering.rename('cluster'), k.degree],1)
clusters = clusters[clusters['cluster'].notna()]
clusters['norm_deg'] = ((clusters['degree'] - clusters['degree'].min())/(clusters['degree'].max() - clusters['degree'].min()))
clusters = pd.concat([clusters, mapping_putative.reindex(clusters.index)], 1)

In [ ]:
temp = supp1.parse('Metadata and PFTs', index_col = 0)
metadata = temp.iloc[0:, 0:2]
metadata['Conditions'] = ["%s (%s)" % (metadata['Treatment'][i], metadata['Day'][i]) if metadata['Treatment'][i] != 'Control' else 'Control' for i in metadata.index]
PFT = temp.iloc[0:, 2:]
norm_pft = zscore(PFT).T.rename(columns = metadata['Conditions'].to_dict())

data = supp3.parse('Putative Metabolomics', index_col = 0)[metadata.index]

mapping_putative = supp3.parse('Putative Mapping', index_col = 0)

## Remove data that were detected in less than 2 samples per group
remove = data[(pd.concat([data.T.notna(),metadata['Conditions']],1).groupby('Conditions').sum() < 2).sum() > 0].index.tolist()
removed_mets = mapping_putative.reindex(remove)
data_putative = data[~data.index.isin(remove)]

Control = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Control'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))
Bleo_7 = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Bleomycin (7 Days)'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))
Bleo_14 = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Bleomycin (14 Days)'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))
Bleo_21 = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Bleomycin (21 Days)'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))
Bleo_28 = impute_knn(data_putative.T.reindex(metadata[metadata['Conditions'] == 'Bleomycin (28 Days)'].index).dropna(how = 'all').dropna(how = 'all', axis = 1))


data_putative_imputed = pd.concat([Control, Bleo_7, Bleo_14, Bleo_21, Bleo_28]).T.dropna()

norm_data_putative_imputed = zscore(data_putative_imputed.T).T.rename(columns = metadata['Conditions'].to_dict())

In [ ]:
x = []
for cl in sorted(clusters['cluster'].unique()):
    sel_data = norm_data_putative_imputed.reindex(clusters[clusters['cluster'] == cl].index).rename(columns = metadata['Conditions'].to_dict())#.T#.reset_index().groupby('index').mean().T.rename(columns = days)
    row_linkage = linkage(distance.pdist(sel_data.fillna(0)), method='average')
    dn = dendrogram(row_linkage, no_plot = True)['leaves']
    temp = sel_data.iloc[dn].index.tolist()
    x.append(temp)

In [ ]:
days = {'Control': 0, 'Bleomycin (7 Days)': 7, 'Bleomycin (14 Days)': 14, 'Bleomycin (21 Days)': 21, 'Bleomycin (28 Days)': 28}
x_interp = np.linspace(days['Control'], days['Bleomycin (28 Days)'], 100)
color_mapping = dict(zip(sorted(clusters['cluster'].unique()),sns.color_palette("tab10").as_hex()[::-1]))

cluster_color = pd.Series(clusters['cluster']).map(color_mapping)
sel_data = norm_data_putative_imputed.reindex(sum(x,[]))
cm = sns.clustermap(sel_data.fillna(0), figsize=(12.63, 10),  yticklabels = 0, xticklabels = 0, center = 0, cmap = 'RdYlBu_r', row_colors = cluster_color, row_cluster = False, col_cluster = False)
cm.cax.set_visible(False)
cm.gs.update(left=0.05, right=0.25)
cm.ax_heatmap.vlines(metadata.groupby('Conditions').count()['Treatment'].cumsum().tolist(), ymin = 0, ymax = sel_data.shape[0], color = 'gray')
cm.ax_heatmap.hlines(np.cumsum([len(i) for i in x]), xmin = 0, xmax = sel_data.shape[1], color = 'black')
cm.ax_heatmap.set_xlabel('Days', fontweight = 'bold', fontsize = 20)
cm.ax_heatmap.set_ylabel('', fontweight = 'bold', fontsize = 20)
cm.ax_heatmap.set_xticks((metadata.groupby('Conditions').count().cumsum() - metadata.groupby('Conditions').count()/2)['Treatment'].tolist())
cm.ax_heatmap.set_xticklabels([0, 7, 14, 21, 28], fontweight = 'bold', fontsize = 15)

gs = GridSpec(int(clusters['cluster'].max()+2),5, top = .95, left = 0.26, hspace = 0.2, 
              wspace = 0.21,width_ratios=[.5,0.1, .5,  1, 0.25])


for cl in sorted(clusters['cluster'].unique().astype(int)):
    sel_ax = cm.fig.add_subplot(gs[cl+1, 0])
    sel_data = norm_data_putative_imputed.reindex(clusters[clusters['cluster'] == cl].index).rename(columns = metadata['Conditions'].to_dict()).T.reset_index().groupby('index').mean().T.rename(columns = days).dropna()[[0, 7, 14, 21, 28]]
    y_interp = interp1d(list(days.values()), list(sel_data.values), kind = 'quadratic')(x_interp)
    data_interp = pd.DataFrame(y_interp, columns = x_interp).unstack().reset_index().groupby('level_0').mean().reset_index()
    temp_max = data_interp.copy()[0]
    g = sns.lineplot(data = data_interp, x = 'level_0', y = 0, ax = sel_ax, color = color_mapping[cl])
    sel_data = norm_pft.reindex(clusters[clusters['cluster'] == cl].index).rename(columns = metadata['Conditions'].to_dict()).T.reset_index().groupby('index').mean().T.rename(columns = days).dropna()[[0, 7, 14, 21, 28]]
    
    if sel_data.shape[0] > 0:
        y_interp = interp1d(list(days.values()), list(sel_data.values), kind = 'quadratic')(x_interp)
        temp_max = pd.concat([temp_max, data_interp.copy()[0]])
        data_interp = pd.DataFrame(y_interp, columns = x_interp).unstack().reset_index().groupby('level_0').mean().reset_index()
        g = sns.lineplot(data = data_interp, x = 'level_0', y = 0, ax = sel_ax, color = color_mapping[cl], linestyle='--')
    if cl == clusters['cluster'].max():
        g.set_xlabel('Days', fontweight = 'bold', fontsize = 20)
        g.set_xticks(list(days.values()))
        g.set_xticklabels(list(days.values()), fontweight = 'bold', fontsize = 15)
    else:
        g.set_xlabel('')
        g.set_xticks([])
    temp_ylim = tuple(g.get_ylim())
    g.vlines([7, 14, 21], ymin = g.get_ylim()[0], ymax = g.get_ylim()[1], color = 'lightgray')
    g.set_ylim(temp_ylim)
    g.set_yticks([])
    g.set_ylabel('')
    g.set_xlim((0,28))
    
for cl in sorted(clusters['cluster'].unique().astype(int)):
    sel_ax = cm.fig.add_subplot(gs[cl+1, 1])
    topx = 5
    temp = clusters[clusters['cluster'] == cl].sort_values('degree', ascending = False).iloc[0:topx].reset_index()
    temp['index'] = temp['Compound name']
    sns.scatterplot(data = temp, x = 'degree', y = 'index', ax = sel_ax, color = color_mapping[cl], size = 'norm_deg', legend = False)
    sel_ax.yaxis.tick_right()
    sel_ax.set_yticklabels([i for i in temp['index']], fontsize = 15)
    sel_ax.set_xticks([0])
    if cl == clusters['cluster'].max():
        sel_ax.set_xlabel('Top Nodes\n(Degree)', fontweight = 'bold', fontsize = 20)
        sel_ax.set_xticks([0])
        sel_ax.set_xticklabels([], fontweight = 'bold', fontsize = 15)
        sns.despine(ax = sel_ax, left = False, bottom = False, right = True, top = True)
    else:
        sel_ax.set_xlabel('')
        sel_ax.set_xticks([])
        sns.despine(ax = sel_ax, left = False, bottom = True, right = True, top = True)
    sel_ax.set_ylabel('')
    sel_ax.tick_params(right=False, bottom=False)

sel_term = [] 
for cl in sorted(clusters['cluster'].unique().astype(int)):
    sel_ax = cm.fig.add_subplot(gs[cl+1, 3])
    print(cl)
    met_list = clusters[(clusters['cluster'] == cl)].index.tolist()
    y = []
    for pw in list(pathways.keys()):
        hist_mets = mapping_putative.reindex(clusters['cluster'].reindex(list(set(met_list).intersection(set(pathways[pw])))).index)['Compound name']
        hits = len(hist_mets)
        num_ofpw = len(set(sum(list(pathways.values()), [])))
        num_ofmets_inpw = len(set(pathways[pw]))
        total_mets = len(set(met_list))
        pval = 1-hypergeom.cdf(hits,num_ofpw-num_ofmets_inpw,total_mets,num_ofmets_inpw)
        y.append([pw, hits, num_ofmets_inpw, ';'.join(list(hist_mets)), pval])
    y = pd.DataFrame(y, columns = ['Pathways', '# of Hits', 'Total Mets', 'Hits Metabolites', 'pval']).set_index('Pathways')
    y = y[(y['# of Hits'] > 0) & (y['Total Mets'] > 1)]
    y = y[~y.index.isin(['Prostate cancer', 'Amyotrophic lateral sclerosis (ALS)', 'Photosynthesis'])]

    temp = y.sort_values('pval').iloc[0:5].reset_index()
    temp['-Log10(P)'] = -np.log10(temp['pval'] + 1E-2)
    temp['color'] = '#a50000'
    sel_term.append(temp['Pathways'].tolist())
    sns.barplot(data = temp, x = '-Log10(P)', y = 'Pathways', color = color_mapping[cl], ax = sel_ax)
    
    sel_ax.yaxis.tick_right()
    sel_ax.set_yticklabels([i for i in temp['Pathways']], fontsize = 15)
    sel_ax.set_xticks([0])
    if cl == max(clusters['cluster'].unique()):
        sel_ax.set_xlabel('-Log10(P-Adj)', fontweight = 'bold', fontsize = 20)
        sel_ax.set_xticks([0])
        sel_ax.set_xticklabels([], fontweight = 'bold', fontsize = 15)
        sns.despine(ax = sel_ax, left = False, bottom = False, right = True, top = True)
    else:
        sel_ax.set_xlabel('')
        sel_ax.set_xticks([])
        sns.despine(ax = sel_ax, left = False, bottom = True, right = True, top = True)
    sel_ax.set_ylabel('')
    sel_ax.tick_params(right=False, bottom=False)

plt.savefig('Figures/Figure5B.pdf')

## Figure 5D-E

### Integrating GCN and MCN

Using the enriched subsystems from Mouse Metabolic Model, retrieved from https://metabolicatlas.org/ on September 9, 2022

In [ ]:
temp = supp1.parse('Metadata and PFTs', index_col = 0)
metadata = temp.iloc[0:, 0:2]
metadata['Conditions'] = ["%s (%s)" % (metadata['Treatment'][i], metadata['Day'][i]) if metadata['Treatment'][i] != 'Control' else 'Control' for i in metadata.index]
tpm = supp1.parse('TPM', index_col = 0)[metadata.index]
mapping_putative = supp3.parse('Putative Mapping', index_col = 0)

In [ ]:
mat = loadmat('lib/Mouse-GEM.mat')['mouseGEM']

stoc_mat = pd.DataFrame.sparse.from_spmatrix(mat['S'][0][0])
fin = []
for i in range(len(mat['subSystems'][0][0])):
    if (len(mat['grRules'][0][0][i][0]) > 0) & (stoc_mat[i].sum() != 0):
        genes = mat['grRules'][0][0][i][0][0].replace('(', '').replace(')', '').replace('and', 'or').split(' or ')
        sub = mat['subSystems'][0][0][i][0][0][0][0]
        met = []
        for j in stoc_mat[i][stoc_mat[i] != 0].index:
            if len(mat['metHMDBID'][0][0][j][0]) > 0:
                met.append(mat['metHMDBID'][0][0][j][0][0])
            else:
                pass
        met = list(set(met))
        fin.append([sub, genes, met])
GMT_genes = pd.DataFrame(fin, columns = ['subsystem', 'genes', 'hmdb']).groupby('subsystem')['genes'].apply(lambda x: list(set(sum(x,[])))).to_dict()
GMT_mets = pd.DataFrame(fin, columns = ['subsystem', 'genes', 'hmdb']).groupby('subsystem')['hmdb'].apply(lambda x: list(set(sum(x,[])))).to_dict()

In [ ]:
k = pickle.load(open('Networks/ReadyObjects/GCN.pkl', 'rb'))

clusters_GCN = pd.concat([k.clustering.rename('cluster'), k.degree],1)
clusters_GCN = clusters_GCN[clusters_GCN['cluster'].notna()]

k = pickle.load(open('Networks/ReadyObjects/MCN.pkl', 'rb'))
mapping_putative = supp3.parse('Putative Mapping', index_col = 0)

clusters_MCN = pd.concat([k.clustering.rename('cluster'), k.degree],1)
clusters_MCN = clusters_MCN[clusters_MCN['cluster'].notna()]
clusters_MCN = pd.concat([clusters_MCN, mapping_putative.reindex(clusters_MCN.index)], 1)

lst = []
for cl in sorted(clusters_MCN['cluster'].unique()):
    temp = clusters_MCN[clusters_MCN['cluster'] == cl]['HMDB ID'].dropna().tolist()
    for i in temp:
        if i.count('HMDB') == 0:
            pass
        elif i.count('HMDB') > 1:
            for j in i.replace(',','\n').split('\n'):
                lst.append([j.replace('HMDB00','HMDB'), 'M-%d' % cl])
        else:
            lst.append([i.replace('\n','').replace('HMDB00','HMDB'), 'M-%d' % cl])
hmdb2cl = pd.DataFrame(lst, columns = ['index','cluster']).drop_duplicates()
temp = clusters_GCN.reset_index()[['index', 'cluster']]
temp['cluster'] = ['G-%d' % i for i in temp.cluster]

cl_mapping = pd.concat([hmdb2cl, temp]).drop_duplicates()

In [ ]:
gene_enrich = pd.Series()
for cl in sorted(clusters_GCN['cluster'].unique()):
    lst = clusters_GCN[clusters_GCN['cluster'] == cl].index.tolist()
    enr = gp.enrichr(gene_list=lst,
             gene_sets=GMT_genes,
             background = tpm.shape[0],
             outdir='test/enrichr_kegg2',
             cutoff=0.5,
             verbose=False)
    temp = enr.res2d.sort_values('Term', ascending = True)#.iloc[0:5]
    temp2 = temp[temp['P-value']<0.05][['Term','P-value']].set_index('Term')
    temp2['gene'] = 'G-%d' % cl
    gene_enrich = pd.concat([gene_enrich, temp2['gene']])

In [ ]:
met_enrich = pd.Series()
for cl in sorted(clusters_MCN['cluster'].unique()):
    temp = clusters_MCN[clusters_MCN['cluster'] == cl]['HMDB ID'].dropna().tolist()
    lst = []
    for i in temp:
        if i.count('HMDB') == 0:
            pass
        elif i.count('HMDB') > 1:
            for j in i.replace(',','\n').split('\n'):
                lst.append(j.replace('HMDB00','HMDB'))
        else:
            lst.append(i.replace('\n','').replace('HMDB00','HMDB'))
    y = []
    for pw in list(GMT_mets.keys()):
        hist_mets = set(lst).intersection(GMT_mets[pw])
        hits = len(hist_mets)
        num_ofpw = len(set(sum(list(GMT_mets.values()), [])))
        num_ofmets_inpw = len(set(GMT_mets[pw]))
        total_mets = len(set(lst))
        pval = 1-hypergeom.cdf(hits,num_ofpw-num_ofmets_inpw,total_mets,num_ofmets_inpw)
        y.append([pw, hits, num_ofmets_inpw, ';'.join(list(hist_mets)), pval])
    y = pd.DataFrame(y, columns = ['Pathways', '# of Hits', 'Total Mets', 'Hits Metabolites', 'pval']).set_index('Pathways')
    y = y[(y['# of Hits'] > 1) & (y['Total Mets'] > 1) & (y['pval'] < 0.05)]
    y['met'] = 'M-%d' % cl
    met_enrich = pd.concat([met_enrich, y['met']])

In [ ]:
intersection = pd.DataFrame(gene_enrich).reset_index().merge(pd.DataFrame(met_enrich).reset_index(), on = 'index')
## Use this file to generate chord diagram (Figure 6D)
intersection[['0_y','0_x']].drop_duplicates().to_csv('Networks/ION/G2M.txt', sep = '\t', index = False)

## Combine the following files in Cytoscape to generate Figure 6E
intersection[['index', '0_x']].to_csv('Networks/ION/G2PW.txt', sep = '\t', index = False)
intersection[['index', '0_y']].to_csv('Networks/ION/M2PW.txt', sep = '\t', index = False)

In [ ]:
### R script used for figure 6D
# library(circlize)

# data = read.csv('Networks/ION/G2M.txt', sep = '\t')
# adjacencyData <- with(data, table(X0_y, X0_x))

# grid.col = c(
#   'M-0' = "#17becf", 'G-0' = "#17becf",
#   'M-1' = "#bcbd22", 'G-1' = "#bcbd22",
#   'M-2' = "#7f7f7f", 'G-2' = "#7f7f7f",
#   'M-3' = "#e377c2", 'G-3' = "#e377c2",
#   'G-4' = "#8c564b"
# )

# circos.par(start.degree = 90)
# chordDiagram(adjacencyData, transparency = 0.25,
#              annotationTrack = c("name", "grid"),
#              order = c('M-0', 'M-1', 'M-2', 'M-3',
#                        'G-4', 'G-3', 'G-2', 'G-1', 'G-0'),
#              grid.col = grid.col#, col = data$link
# )
# circos.clear()

## Figure 6B

Use Supplementary Data 5

OR

1. Download the data from PRJNA358081, and PRJNA388978.
2. Perform differential expression analysis using the code above

Excluded outliers:
* PRJNA358081: 'SRR5120909','SRR5120924','SRR5120930'
* PRJNA388978: Use only the HC and IPFs

To draw the Figure 6B, use the selected genes below ("sel_genes". Criteria: has to be significant in at least 2 timepoints and in all human IPF patients)

In [ ]:
res = pd.DataFrame()

deseq = pd.ExcelFile('StatisticalAnalysis/Transcriptomics_TimeSeries/DifferentialExpression_ALL.xlsx')
for i in deseq.sheet_names[0:]:
    name = i.replace('WT_','').replace('Bleo_','Bleo-').replace('_',' vs ')
    temp = deseq.parse(i, index_col = 0)
    temp = temp[temp['P-ADJ'] < 0.05]['DIRECTION'].rename(name)
    res = pd.concat([res, temp],1)
res.index = res.index.str.upper()

for i in supp5.sheet_names[0:3]:
    name = i.replace('_',' vs ')
    temp = supp5.parse(i, index_col = 0)
    temp = temp[temp['P-ADJ'] < 0.05]['DIRECTION'].rename(name)
    res = pd.concat([res, temp],1)

In [ ]:
res_sel = res.reindex(res.iloc[0:, 0:-2].dropna(thresh = 2).index)
sel_genes = res_sel[res_sel['GSE92592'].notna() & res_sel['GSE99621'].notna()]#.index

## Figure 6C-D

In [ ]:
clust = pd.read_csv('../Revision/Cluster_Human_new.txt', sep = '\t', index_col = 0)['0'].replace(2,1)
tpm = supp1.parse('TPM', index_col = 0)[metadata.index]

KEGG = {i.split('\t')[0]:[j for j in i.rstrip('\n').split('\t')[2:] if j != ''] for i in open('lib/KEGG_2019_Mouse.gmt', 'r').readlines()}
exclude_KEGG = ['Hepatitis B', 'Human T-cell leukemia virus 1 infection', 'Regulation of actin cytoskeleton', 'Measles',
           'MicroRNAs in cancer', 'Pathways in cancer', 'Influenza A', 'Oocyte meiosis', 'Progesterone-mediated oocyte maturation',
           'Ribosome biogenesis in eukaryotes', 'Amoebiasis', 'Small cell lung cancer', 'Epstein-Barr virus infection', 'Human papillomavirus infection',
           'Pertussis', 'Staphylococcus aureus infection', 'Proteoglycans in cancer', 'Leishmaniasis', 'Legionellosis', 'Viral carcinogenesis', 
           'Chagas disease (American trypanosomiasis)', 'Toxoplasmosis', 'Malaria', 'Metabolism of xenobiotics by cytochrome P450', 'Chronic myeloid leukemia',
           'Prion diseases', 'Inflammatory bowel disease (IBD)', 'Kaposi sarcoma-associated herpesvirus infection', 'Parkinson disease', 'Hepatitis C', 'Systemic lupus erythematosus',
           'Tuberculosis', 'Hepatitis B', 'Measles', 'Dilated cardiomyopathy (DCM)', 'Hematopoietic cell lineage', 'Adrenergic signaling in cardiomyocytes',
           'Epstein-Barr virus infection', 'Drug metabolism', 'Salivary secretion', 'Thermogenesis', 'Glioma', 'Human cytomegalovirus infection', 'Melanoma', 'Chemical carcinogenesis',
            'African trypanosomiasis', 'Collecting duct acid secretion', 'African trypanosomiasis', 'Vascular smooth muscle contraction', 'Fluid shear stress and atherosclerosis',
            'Rheumatoid arthritis', 'Allograft rejection', 'Long term depression'
          ]

exclude_GO = ['',
             
             ]

color_mapping = dict(zip(sorted(clust.unique()),sns.color_palette("tab10").as_hex()))


fig, axs = plt.subplots(figsize = (2, 3),nrows = len(clust.unique().tolist()), ncols = 1, sharex=False
                        , gridspec_kw={'height_ratios':[5, 5]})

for j, sel_ax in enumerate(axs):
    print()
    gl = [ha.upper() for ha in clust[clust == j].index.tolist()]
    enr = gp.enrichr(gene_list=gl,
             gene_sets=KEGG,
             background = tpm.shape[0],
             outdir='test/enrichr_kegg2',
             cutoff=1,
             verbose=False)
    temp = enr.res2d[~(
        enr.res2d['Term'].isin(exclude_KEGG) |
        (enr.res2d['Term'].str.contains('isease')) |
         (enr.res2d['Term'].str.contains('arcinoma')) |
          (enr.res2d['Term'].str.contains('ancer')) |
        (enr.res2d['Term'].str.contains('thyroid')) |
        (enr.res2d['Term'].str.contains('yndrome')) |
        (enr.res2d['Term'].str.contains('strogen')) |
        (enr.res2d['Term'].str.contains('elanoge')) |
        (enr.res2d['Term'].str.contains('iabet')) |
        (enr.res2d['Term'].str.contains('ysosome')) |
        (enr.res2d['Term'].str.contains('nfection')) |
        (enr.res2d['Term'].str.contains('ntestin')) |
        (enr.res2d['Term'].str.contains('cardi')) |
        (enr.res2d['Term'].str.contains('mmunodef'))
    )].sort_values('Adjusted P-value', ascending = True)#.iloc[0:5]#.iloc[[2,3,4]]['Term'].tolist()
    temp2 = temp[['Term','P-value','Adjusted P-value']]
    temp2 = temp2[temp2['Adjusted P-value'] < 0.05]
    print(temp2.shape)
    temp2['Term'] = temp2['Term'].str.replace('endoplasmic reticulum', 'ER')
    temp2['-Log10(P-Adj)'] = -np.log10(temp2['Adjusted P-value'])
    sns.barplot(data = temp2, x = '-Log10(P-Adj)', y = 'Term', ax = sel_ax, color = color_mapping[j])
    sel_ax.yaxis.tick_right()
    sel_ax.set_yticklabels([i for i in temp2['Term']], fontsize = 15)
#     sel_ax.set_xticks([])
    sel_ax.set_ylabel('DEG-%d (%d Genes)' % (j, len(gl)), fontsize = 15, fontweight = 'bold')
    sel_ax.tick_params(right = False)
    if j == clust.max():
        sns.despine(ax = sel_ax)
        sel_ax.set_xlabel('-Log10(P-Adj)', fontsize = 15, fontweight = 'bold')
    else:
        sns.despine(ax = sel_ax, bottom = False)
        sel_ax.set_xlabel('')
plt.savefig('Figures/Figure6C-D_funct.pdf')

## Figure 7A, C, D

This was done via iDREM. Please refer to iDREM tutorial and Supplementary Table 6

Figure 7C (and Supplementary Figure 5) was regenerated using Pysankey package, but can be generated directly using iDREM.

Final iDREM model and visualization can be found in the github repository

## Figure 7B

Use "iDREM_Results/GCN_iDREM_edges.txt" generated below as the input to cytoscape

In [ ]:
k = pickle.load(open('Networks/ReadyObjects/GCN.pkl', 'rb'))
clust = k.clustering
clust.index = clust.index.str.upper()
iDREM_mapping = supp6.parse('Gene Tracks', index_col = 0)['Tracks']

In [ ]:
num_ofpw = clust.shape[0]
y = []
for cl in sorted(clust.unique()):
    temp = clust[clust == cl]
    for node in iDREM_mapping.unique():
        hits_an = set(temp.index.str.upper()).intersection(iDREM_mapping[iDREM_mapping == node].index)
        hits = len(hits_an)
        num_member_inpw = iDREM_mapping[iDREM_mapping == node].shape[0]
        pval = 1-hypergeom.cdf(hits,num_ofpw-num_member_inpw, temp.shape[0],num_member_inpw)
        y.append([node, 'G-%s' % cl, pval])
results_cl = pd.DataFrame(y, columns = ['node', 'cl', 'pval'])
results_cl['padj']=multipletests(results_cl['pval'],method='fdr_bh')[1]
results_cl = results_cl[results_cl['padj'] < 0.05].sort_values(['node'], ascending = False)
results_cl['-Log10(P-Adj)'] = -np.log1p(results_cl['padj'])
results_cl.to_csv('iDREM_Results/GCN_iDREM_edges.txt', sep = '\t')

## Differential Expression & Functional Analysis of CB1R KO

In [ ]:
temp = supp5.parse('Metadata and PFTs', index_col = 1)
metadata = temp.iloc[0:, 0:2]
count = supp5.parse('Counts', index_col = 0)[metadata.index]

In [ ]:
res_path='StatisticalAnalysis/Transcriptomics_CB1RKO/'
k=DESeq_Piano(res_path)
k.DEseq(count,metadata['Study group'].str.replace('+', '_').str.replace(' ', '').str.replace('-', ''))
k.loadGSC(KEGG='lib/KEGG_2019_Mouse.gmt')

cond2='Control'
cond1='wildtype_Bleomycin'
k.DEseq_Compare(cond1,cond2,save=True)
k.PIANO(cond1,cond2,save=True)

cond2='wildtype_Bleomycin'
cond1='GlobalCB1RKO_Bleomycin'
k.DEseq_Compare(cond1,cond2,save=True)
k.PIANO(cond1,cond2,save=True)

k.summarizeDEseq()
k.summarizePiano()


## Figure 8A

In [ ]:
temp = supp7.parse('Metadata and PFTs', index_col = 1)
metadata = temp.iloc[0:, 0:1]
PFT = temp.iloc[0:, 1:]

In [ ]:
fig, axs = plt.subplots(figsize = (13, 2.5),nrows = 1, ncols = 5, sharex=True)
# axs = sum([list(i) for i in axs], [])

for num, pft in enumerate(PFT.columns[1:][::-1]):
    temp = pd.concat([PFT[pft], metadata],1).reset_index()
    order = ['Control','wild-type+ Bleomycin', ' Global CB1R KO + Bleomycin']
    box_pairs = [
        ('Control', 'wild-type+ Bleomycin'),
        ('Control', ' Global CB1R KO + Bleomycin'),
        (' Global CB1R KO + Bleomycin', 'wild-type+ Bleomycin')
    ]
    
    pval_sel = [
        f_oneway(temp[temp['Study group'] == 'wild-type+ Bleomycin'][pft], 
             temp[temp['Study group'] == 'Control'][pft])[1],
        f_oneway(temp[temp['Study group'] == 'Control'][pft], 
             temp[temp['Study group'] == ' Global CB1R KO + Bleomycin'][pft])[1],
        f_oneway(temp[temp['Study group'] == 'wild-type+ Bleomycin'][pft],
             temp[temp['Study group'] == ' Global CB1R KO + Bleomycin'][pft])[1]
    ]

    sns.boxplot(data = temp, x = 'Study group', y = pft, order=order, ax = axs[num])
    add_stat_annotation(data = temp, x = 'Study group', y = pft, order=order, ax = axs[num],
                        box_pairs=box_pairs,
                        perform_stat_test=False, pvalues=pval_sel,
                        test=None, text_format='star', loc='inside', verbose=0, fontsize = 15)
    axs[num].set_xticklabels(axs[num].get_xticklabels(), fontsize = 15, fontweight = 'bold', rotation = 30, ha = 'right')
    axs[num].set_xlabel('')
    axs[num].set_ylabel('')
    if num < 3:
        axs[num].set_yticks([0.5, 1, 1.5])
    axs[num].set_yticklabels(['%d' % j if int(j) == j else '%.1f' % j for j in axs[num].get_yticks()], fontsize = 15)
    axs[num].set_title('%s' % (pft.replace(' (PFT)', '')), fontsize = 15, fontweight = 'bold')
    plt.subplots_adjust(wspace = 0.3)
plt.savefig('Figures/Figure8A.pdf', transparent = True)

## Figure 8B

In [ ]:
hyp = supp7.parse('Hydroxyproline')
qpcr = supp7.parse('qPCR', index_col = 0)

In [ ]:
fig, axs = plt.subplots(figsize = (13, 2.5),nrows = 1, ncols = 5, sharex=True)

order = ['Control','wild-type+ Bleomycin', ' Global CB1R KO + Bleomycin']
box_pairs = [
    ('Control', 'wild-type+ Bleomycin'),
    ('Control', ' Global CB1R KO + Bleomycin'),
    (' Global CB1R KO + Bleomycin', 'wild-type+ Bleomycin')
]

var = 'Hydroxyproline'
num = 0

pval_sel = [
    f_oneway(hyp[hyp['Unnamed: 0'] == 'wild-type+ Bleomycin']['Hydroxyproline'], 
         hyp[hyp['Unnamed: 0'] == 'Control']['Hydroxyproline'])[1],
    f_oneway(hyp[hyp['Unnamed: 0'] == 'Control']['Hydroxyproline'], 
         hyp[hyp['Unnamed: 0'] == ' Global CB1R KO + Bleomycin']['Hydroxyproline'])[1],
    f_oneway(hyp[hyp['Unnamed: 0'] == 'wild-type+ Bleomycin']['Hydroxyproline'],
         hyp[hyp['Unnamed: 0'] == ' Global CB1R KO + Bleomycin']['Hydroxyproline'])[1]
]
sns.boxplot(data = hyp, x = 'Unnamed: 0', y = var, order=order, ax = axs[num])
add_stat_annotation(data = hyp, x = 'Unnamed: 0', y = var, order=order, ax = axs[num],
                    box_pairs=box_pairs,
                    perform_stat_test=False, pvalues=pval_sel,
                    test=None, text_format='star', loc='inside', verbose=0, fontsize = 15)
axs[num].set_xticklabels(axs[num].get_xticklabels(), fontsize = 15, fontweight = 'bold', rotation = 30, ha = 'right')
axs[num].set_xlabel('')
axs[num].set_ylabel('')
axs[num].set_yticklabels(['%d' % j if int(j) == j else '%.1f' % j for j in axs[num].get_yticks()], fontsize = 15)
axs[num].set_title('%s' % var, fontsize = 15, fontweight = 'bold')
plt.subplots_adjust(wspace = 0.3)


for num, var in enumerate(qpcr.columns[1:-1]):
    num = num + 1
    fc = pd.concat([qpcr[['conds']+[var]]],1).dropna()
    order = ['Control','WT-Bleo', 'CB1R KO-Bleo']
    box_pairs = [
        ('Control', 'WT-Bleo'),
        ('CB1R KO-Bleo', 'Control'),
        ('CB1R KO-Bleo', 'WT-Bleo')
                ]
    
    pval_sel = [
        f_oneway(fc[fc['conds'] == 'WT-Bleo'][var], 
             fc[fc['conds'] == 'Control'][var])[1],
        f_oneway(fc[fc['conds'] == 'Control'][var], 
             fc[fc['conds'] == 'CB1R KO-Bleo'][var])[1],
        f_oneway(fc[fc['conds'] == 'WT-Bleo'][var], 
             fc[fc['conds'] == 'CB1R KO-Bleo'][var])[1],
    ]

    sns.boxplot(data = fc, x = 'conds', y = var, order=order, ax = axs[num])
    add_stat_annotation(data = fc, x = 'conds', y = var, order=order, ax = axs[num],
                        box_pairs=box_pairs,
                        perform_stat_test=False, pvalues=pval_sel,
                        test=None, text_format='star', loc='inside', verbose=0, fontsize = 15)
    axs[num].set_xticklabels(axs[num].get_xticklabels(), fontsize = 15, fontweight = 'bold', rotation = 30, ha = 'right')
    axs[num].set_xlabel('')
    axs[num].set_ylabel('')
    if num == 3:
        axs[num].set_yticks([1, 2])
        axs[num].set_ylim([0.75, 2.5])
    axs[num].set_yticklabels(['%d' % j if int(j) == j else '%.1f' % j for j in axs[num].get_yticks()], fontsize = 15)
    axs[num].set_title('%s' % var, fontsize = 15, fontweight = 'bold')
    plt.subplots_adjust(wspace = 0.3)
plt.savefig('Figures/Figure8B-C.pdf', transparent = True)

## Figure 8D

In [ ]:
piano = pd.ExcelFile('StatisticalAnalysis/Transcriptomics_TimeSeries/KEGG_ALL.xlsx')
res = pd.DataFrame()
for i in ['Bleomycin_14Days_Control']:
    temp = piano.parse(i, index_col = 0)
    temp = temp['Stats'].rename(i)
    res = pd.concat([res, temp],1)

piano = pd.ExcelFile('StatisticalAnalysis/Transcriptomics_CB1RKO/KEGG_ALL.xlsx')
for i in piano.sheet_names[0:]:
    temp = piano.parse(i, index_col = 0)
    temp = temp['Stats'].rename(i)
    res = pd.concat([res, temp],1)
    
res = res[[
    'Bleomycin_14Days_Control', 'wildtype_Bleomycin_Control', 'GlobalCB1RKO_Bleomycin_wildtype'
]]
    
res = res[[
    'Bleomycin_14Days_Control', 'wildtype_Bleomycin_Control', 'GlobalCB1RKO_Bleomycin_wildtype'
]]

In [ ]:
exclude_KEGG = ['Hepatitis B', 'Human T-cell leukemia virus 1 infection', 'Regulation of actin cytoskeleton', 'Measles',
           'MicroRNAs in cancer', 'Pathways in cancer', 'Influenza A', 'Oocyte meiosis', 'Progesterone-mediated oocyte maturation',
           'Ribosome biogenesis in eukaryotes', 'Amoebiasis', 'Small cell lung cancer', 'Epstein-Barr virus infection', 'Human papillomavirus infection',
           'Pertussis', 'Staphylococcus aureus infection', 'Proteoglycans in cancer', 'Leishmaniasis', 'Legionellosis', 'Viral carcinogenesis', 
           'Chagas disease (American trypanosomiasis)', 'Toxoplasmosis', 'Malaria', 'Metabolism of xenobiotics by cytochrome P450', 'Chronic myeloid leukemia',
           'Prion diseases', 'Inflammatory bowel disease (IBD)', 'Kaposi sarcoma-associated herpesvirus infection', 'Parkinson disease', 'Hepatitis C', 'Systemic lupus erythematosus',
           'Tuberculosis', 'Hepatitis B', 'Measles', 'Dilated cardiomyopathy (DCM)', 'Hematopoietic cell lineage', 'Adrenergic signaling in cardiomyocytes',
           'Epstein-Barr virus infection', 'Drug metabolism', 'Salivary secretion', 'Thermogenesis', 'Glioma', 'Human cytomegalovirus infection', 'Melanoma', 'Chemical carcinogenesis',
            'African trypanosomiasis', 'Collecting duct acid secretion', 'African trypanosomiasis', 'Vascular smooth muscle contraction', 'Fluid shear stress and atherosclerosis',
            'Rheumatoid arthritis', 'Allograft rejection', 'Long term depression'
          ]

In [ ]:
res_sel = res[~(
    res.index.isin(exclude_KEGG) |
    (res.index.str.contains('isease')) |
     (res.index.str.contains('arcinoma')) |
      (res.index.str.contains('ancer')) |
    (res.index.str.contains('thyroid')) |
    (res.index.str.contains('yndrome')) |
    (res.index.str.contains('strogen')) |
    (res.index.str.contains('elanoge')) |
    (res.index.str.contains('iabet')) |
    (res.index.str.contains('ysosome')) |
    (res.index.str.contains('nfection')) |
    (res.index.str.contains('ntestin')) |
    (res.index.str.contains('cardi')) |
    (res.index.str.contains('Cardi')) |
    (res.index.str.contains('nemia')) |
    (res.index.str.contains('mmunodef'))
)]

In [ ]:
temp = res_sel.dropna()
cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#0000a5",'#0000d8',"#FFFAF0",'#d80000',"#a50000"])
g = sns.clustermap(temp.fillna(0), yticklabels = 1, 
               center = 0, cmap = cmap, 
               col_cluster = False, row_cluster = True,
               figsize = (7,10),
               cbar_kws={"orientation": "vertical","shrink": 1, 'label': 'Gene-Set Statistic'}
              )
g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xticklabels(),rotation=30, ha = 'right', fontsize = 15, fontweight = 'bold')
g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_yticklabels(), fontsize = 15, )
g.ax_heatmap.tick_params(right=False, bottom=False)
g.savefig('Figures/Figure8C.pdf')

## Figure 8E

Direction for Bleo+CB1R-KO was based on the hypergeometric test between the clusters and differential expression analysis of CB1R KO vs WildType Bleomycin, as calculated below

In [ ]:
k = pickle.load(open('Networks/ReadyObjects/GCN.pkl', 'rb'))
clust = k.clustering
tpm = supp7.parse('TPM', index_col = 0)

In [ ]:
deseq = pd.ExcelFile('StatisticalAnalysis/Transcriptomics_CB1RKO/DifferentialExpression_ALL.xlsx')

res = pd.DataFrame()
for i in deseq.sheet_names:
    temp = deseq.parse(i, index_col = 0)
    temp = temp[temp['P-ADJ'] < 0.05]['DIRECTION'].rename(i)
    res = pd.concat([res, temp],1)

In [ ]:
num_ofpw = tpm.shape[0]
y = []
for cl in sorted(clust.unique()):
    temp = clust[clust == cl]
    for direction in ['UP','DOWN']:
        hits_an = set(temp.index).intersection(res[res['GlobalCB1RKO_Bleomycin_wildtype'] == direction].index)
        hits = len(hits_an)
        num_member_inpw = res[res['GlobalCB1RKO_Bleomycin_wildtype'] == direction].shape[0]
        pval = 1-hypergeom.cdf(hits,num_ofpw-num_member_inpw, temp.shape[0],num_member_inpw)
        y.append([direction, 'G-%d' % cl, pval])

In [ ]:
results_cl = pd.DataFrame(y, columns = ['node', 'cl', 'pval'])
results_cl['padj']=multipletests(results_cl['pval'],method='fdr_bh')[1]
results_cl = results_cl[results_cl['padj'] < 0.05].sort_values(['node'], ascending = False)
results_cl

## Figure 7D

In [ ]:
res = pd.DataFrame()

for i in ['DEGs 14 Days']:
    name = i.replace('WT_','').replace('Bleo_','Bleo-').replace('_',' vs ')
    temp = supp1.parse(i, index_col = 0)
    temp = temp[temp['P-ADJ'] < 0.05]['L2FC'].rename(name)
    res = pd.concat([res, temp],1)
res.index = res.index.str.upper()

for i in supp5.sheet_names:
        name = i.replace('_',' vs ')
        temp = supp5.parse(i, index_col = 0)
        temp = temp[temp['P-ADJ'] < 0.05]['L2FC'].rename(name)
        res = pd.concat([res, temp],1)


In [ ]:
iDREM_mapping = supp6.parse('Gene Tracks', index_col = 0)['Tracks']
gn = res[np.sign(res).sum(1).abs() == 3].dropna(how = 'all')

In [ ]:
lut = dict(zip(iDREM_mapping.unique(), plt.cm.tab20.colors))
row_colors = iDREM_mapping.map(lut)

In [ ]:
cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#0000a5",'#0000d8',"#FFFAF0",'#d80000',"#a50000"])
temp = gn[gn.index.isin(iDREM_mapping[iDREM_mapping == 'A'].index)]
g = sns.clustermap(temp, cmap = cmap, center = 0, figsize = (3, 5), yticklabels = 1, col_cluster = False, vmin = 0, vmax = 8.2, row_colors = row_colors)
g.savefig('Figures/7E_subsetA.pdf')

In [ ]:
cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#0000a5",'#0000d8',"#FFFAF0",'#d80000',"#a50000"])
temp = gn[gn.index.isin(iDREM_mapping[iDREM_mapping == 'B'].index)]
g = sns.clustermap(temp, cmap = cmap, center = 0, figsize = (3, 12), yticklabels = 1, col_cluster = False, vmin = 0, vmax = 8.2, row_colors = row_colors)
g.ax_heatmap.set_xticks([])
g.savefig('Figures/7E_subsetB.pdf')

## Figure 8H
(Execute Figure 7D first, as this is subsetting the genes there)

In [ ]:
res = pd.DataFrame()

for i in ['DEGs Bleo WT vs Control', 'DEGs Bleo KO vs Bleo WT']:
    name = i.replace('WT_','').replace('Bleo_','Bleo-').replace('_',' vs ')
    temp = supp7.parse(i, index_col = 0)
    temp = temp[temp['P-VALUE'] < 0.05]['L2FC'].rename(name)
    res = pd.concat([res, temp],1)
res.index = res.index.str.upper()

for i in supp5.sheet_names:
        name = i.replace('_',' vs ')
        temp = supp5.parse(i, index_col = 0)
        temp = temp[temp['P-ADJ'] < 0.05]['L2FC'].rename(name)
        res = pd.concat([res, temp],1)

res = res.reindex(gn.index).dropna()

res = res[['GSE92592', 'GSE99621', 'DEGs Bleo WT vs Control', 'DEGs Bleo KO vs Bleo WT']]

In [ ]:
temp = res[res.index.isin(iDREM_mapping.index)]

In [ ]:
lut = dict(zip(iDREM_mapping.unique(), plt.cm.tab20.colors))
row_colors = iDREM_mapping.map(lut)

cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#0000a5",'#0000d8',"#FFFAF0",'#d80000',"#a50000"])
temp = res[res.index.isin(iDREM_mapping[iDREM_mapping == 'A'].index)]

g = sns.clustermap(temp, cmap = cmap, center = 0, figsize = (4, 4), yticklabels = 1, col_cluster = False, vmin = -1.7, vmax = 6.5, row_colors = row_colors)
g.ax_heatmap.set_xticks([])
g.cax.set_visible(False)
g.savefig('Figures/8H_subsetA.pdf')

cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#0000a5",'#0000d8',"#FFFAF0",'#d80000',"#a50000"])
temp = res[res.index.isin(iDREM_mapping[iDREM_mapping == 'B'].index)]

g = sns.clustermap(temp, cmap = cmap, center = 0, figsize = (4, 10), yticklabels = 1, col_cluster = False, vmin = -1.7, vmax = 6.5, row_colors = row_colors)
g.ax_heatmap.set_xticks([])
g.cax.set_visible(False)
g.savefig('Figures/8H_subsetB.pdf')